# BT4103 Data Cleaning
Please read through and let me know if there are any issues with regard to the cleaning.

## Import packages and datasets
I have imported all the packages that I used up here for ease of reference. Please add your own filepath so that you can import the data correctly.

In [3725]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
import unicodedata
import datetime

### Importing dataset
Add filepath below

In [3726]:
data = pd.read_excel("oots-cleaned2.xlsx")
data

validation_data = pd.read_excel("oots-cleaned-unlocked.xlsx")

## Define Helper Functions
Here are the helper functions that I have created for easier readability in the actual code below.

In [3727]:
def OHE(df, col, drop_first=False): #One Hot Encode a column in a df
    dummies = pd.get_dummies(df[col], prefix=col, drop_first=drop_first)
    df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
    legend = {new_col: category 
              for new_col, category in zip(dummies.columns, dummies.columns.str.replace(f"{col}_", "", regex=False))}
    return df, legend


def LabelEncode(df, col): #Label Encode a column in a df
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    legend = dict(zip(le.classes_, le.transform(le.classes_)))
    return df, legend

def inspect_column(df, col, top_n=20): #Get Unique Value information on column in df
    print(f"Column: {col}")
    print(f"Unique values: {df[col].nunique(dropna=False)}")
    print("\nTop value counts:")
    print(df[col].value_counts(dropna=False).head(top_n))

def apply_mapping(df, col, mapping, new_col_suffix="_clean"): #Apply new mapping to column in df
    new_col = col + new_col_suffix
    df[new_col] = df[col].replace(mapping)
    return df

def normalize_text(df, col, to_lower=True, replace_symbols=True, unknown_vals=None): #Normalise text of a column in a df
    s = df[col].astype(str).str.strip()
    if to_lower:
        s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)
    if replace_symbols:
        s = s.str.replace(r"[-_/]", " ", regex=True)
    if unknown_vals:
        s = s.replace(unknown_vals, "0")
    df[col] = s
    return df

def _normalize_text_series(s: pd.Series) -> pd.Series: #Normalise text in series
    s = s.astype(str).map(lambda x: unicodedata.normalize("NFKC", x))
    s = s.str.strip()
    s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)         
    s = s.str.replace(r"\s*/\s*", " / ", regex=True)   
    s = s.str.replace(r"\s*,\s*", ", ", regex=True)    
    s = s.str.strip(" ,")                              
    return s

def _remove_trailing_code_in_parens(name_s: pd.Series, code_s: pd.Series) -> pd.Series: #remove white spaces in ()
    code_up = code_s.astype(str).str.strip().str.upper()
    pattern = r"\(\s*{}\s*\)\s*$"
    out = name_s.copy()
    mask = code_up.notna() & code_up.ne("")
    out.loc[mask] = [
        re.sub(pattern.format(re.escape(c)), "", n, flags=re.IGNORECASE)
        for n, c in zip(out.loc[mask].tolist(), code_up.loc[mask].tolist())
    ]
    return out.str.strip(" ,")

def _choose_canonical_name(name_series: pd.Series) -> str: #Chooses best name
    s = name_series.dropna().astype(str)
    s = s[s.str.strip().ne("").values]
    s = s[s.str.strip().ne("unknown").values]
    if s.empty:
        return "0"
    vc = s.value_counts()
    top_freq = vc.iloc[0]
    candidates = vc[vc.eq(top_freq)].index.tolist()
    return max(candidates, key=len)

def build_operation_legend_and_drop_nature( #Nature cleaning
    df: pd.DataFrame,
    code_col: str = "OPERATION_CODE",
    nature_col: str = "NATURE",
    drop_nature: bool = True,
    keep_title_case_copy: bool = False,
    unknown_tokens = ("0", "na", "n/a", "-", "null", "nan")
):
    if code_col not in df.columns or nature_col not in df.columns:
        raise KeyError(f"Expected columns '{code_col}' and '{nature_col}' in df.")
    work = pd.DataFrame({
        "operation_code": df[code_col].astype(str).str.strip().str.upper(),
        "operation_name_raw": df[nature_col]
    })
    name_norm = _normalize_text_series(work["operation_name_raw"])
    name_norm = name_norm.replace(list(unknown_tokens), "unknown")
    name_clean = _remove_trailing_code_in_parens(name_norm, work["operation_code"])
    tmp = pd.DataFrame({"operation_code": work["operation_code"], "operation_name": name_clean})
    tmp = tmp[tmp["operation_code"].str.len() > 0]
    legend = (
        tmp.groupby("operation_code", as_index=False)["operation_name"]
           .apply(_choose_canonical_name)
           .rename(columns={"operation_name": "operation_name"})
    )
    if keep_title_case_copy:
        legend["operation_name_title"] = legend["operation_name"].str.title()
    df_out = df.copy()
    if drop_nature:
        df_out.drop(columns=[nature_col], inplace=True, errors="ignore")
    return df_out, legend

  
def clean_equipment( #EQUIPMENT cleaning
    df,
    col="EQUIPMENT",
    sep=";",
    tags_to_strip=(r"#nuh",),            
    unknown_vals=("0","na","n/a","-","null","nan",""),
    synonym_map=None         
):
    if synonym_map is None:
        synonym_map = {}
    pattern = r"|".join(fr"{re.escape(tag)}[_-]?" for tag in tags_to_strip)
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(pattern, "", regex=True, flags=re.IGNORECASE)
    )
    df = normalize_text(df, col, to_lower=True, replace_symbols=True, unknown_vals=unknown_vals)
    def _clean_token(tok: str) -> str:
        t = tok.strip()
        if not t: return ""
        t = re.sub(r"\s+", " ", t)
        t = synonym_map.get(t, t)
        if t in ("unknown",): return ""
        return t
    def _process_cell(cell: str) -> str:
        parts = re.split(rf"\s*{re.escape(sep)}\s*", cell) if cell else []
        cleaned = [_clean_token(p) for p in parts]
        cleaned = [c for c in cleaned if c]
        if not cleaned:
            return "unknown"
        cleaned = sorted(set(cleaned))
        return f"{sep} ".join(cleaned)
    df[col] = df[col].apply(_process_cell)
    return df

## General Cleaning
We are dropping index and case number as they are labels, and dropping patient name because it has been completely removed.

In [3728]:
data = data.drop(data.columns[0], axis=1) # drop INDEX
data = data.drop(columns="PATIENT_NAME")
data = data.drop(columns="CASE_NUMBER")
data

,OPERATION_ID,LOCATION,ROOM,BOOKING_DATE,CASE_STATUS,OPERATION_TYPE,EMERGENCY_PRIORITY,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,...,ADMISSION_BED,AOH,BLOOD,IMPLANT,DIAGNOSIS,CANCER_INDICATOR,TRAUMA_INDICATOR,Delay_Reason,Remarks,PATIENT_CODE_OLD
0,588456.0,Main Building OT,MBOR11,10:27:00,Final,Elective,NaN,09:50:00,2011-04-19 09:50:00,09:50:00,...,N06A036,False,NIL,required microscope,Right Breast CA,False,False,Surgeon (e.g. Surgeon not available & etc.),NaN,i8fr0NKEp
1,590736.0,Main Building OT,MBOR05,14:50:00,Final,Elective,NaN,10:40:00,2011-04-19 10:40:00,10:40:00,...,NASWA11,False,NIL,NaN,early pregnancy failure,False,False,NaN,NaN,So30dGuVZ
2,591995.0,ICL,RoomC,18:30:00,Actualised,Elective,NaN,10:55:00,2011-04-19 10:55:00,10:55:00,...,N07B005,False,NIL,NaN,NaN,False,False,NaN,NaN,0t2jmERej
3,590451.0,Main Building OT,MBOR04,11:49:00,Final,Elective,NaN,10:50:00,2011-04-19 10:50:00,10:50:00,...,N041004,False,NIL,Need Eustachian tube ballon,EUSTACHIAN TUBE DISORDER,False,False,NaN,NaN,wDgw8UGdT
4,573666.0,Medical Center OT,MCOR03,09:00:00,Final,Elective,NaN,10:30:00,2011-04-19 10:30:00,10:30:00,...,N02A025,False,NIL,NaN,gall bladder stone,False,False,first case havent finished,first case havent finished,WYQEJDtnk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276858,652260.0,Endoscopy Center,ENDO4,14:30:00,Actualised,Elective,NaN,09:25:00,NaT,09:25:00,...,NaN,False,NIL,NaN,NaN,False,False,NaN,NaN,7W2rCAd5h
276859,667863.0,Main Building OT,MBOR09,13:40:00,Final,Elective,NaN,12:45:00,2019-06-20 12:45:00,12:45:00,...,N052023,False,NIL,ENT Endoscope SetENT DrillENT Neuronavigation,Pituitary Tumour,False,False,Resequencing of cases,NaN,5CUH725Gw
276860,667473.0,Endoscopy Center,ENDO2,16:16:00,Actualised,Elective,NaN,08:30:00,NaT,08:30:00,...,N042041,False,NIL,NaN,Follow up on varices.,False,False,NaN,NaN,g9q0nQP79
276861,665881.0,Medical Center OT,MCOR05,17:52:00,Final,Elective,NaN,09:20:00,2019-06-20 09:20:00,09:20:00,...,NASWC82,False,NIL,NaN,left eye cataract,False,False,NaN,NaN,6RhdSQCfM


## Ensure correct data types
yet to do, still waiting on update from joey regarding the dates

In [3729]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276863 entries, 0 to 276862
Data columns (total 54 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   OPERATION_ID                   276856 non-null  float64       
 1   LOCATION                       276856 non-null  object        
 2   ROOM                           276854 non-null  object        
 3   BOOKING_DATE                   276856 non-null  object        
 4   CASE_STATUS                    276856 non-null  object        
 5   OPERATION_TYPE                 276856 non-null  object        
 6   EMERGENCY_PRIORITY             42137 non-null   object        
 7   PLANNED_PATIENT_CALL_TIME      276856 non-null  object        
 8   PLANNED_PATIENT_FETCH_TIME     208445 non-null  datetime64[ns]
 9   PLANNED_RECEPTION_IN_TIME      276856 non-null  object        
 10  PLANNED_ENTER_OR_TIME          276856 non-null  object        
 11  

## Handle Missing Data
Handling missing data is important to ensure that we do not run into any issues with the EDA, as well as our AI/ML implementations. It is a vital step in data cleaning to ensure that the dataset can be used efficiently and properly.

### Handling optional columns
These columns potentially will be blank as there is nothing to write for some operations, hence we will fill those with 0.

In [3730]:
data["Delay_Reason"].fillna(0, inplace=True)
data["Remarks"].fillna(0, inplace=True)
data["IMPLANT"].fillna(0, inplace=True)
data["EQUIPMENT"].fillna(0, inplace=True)
data["EMERGENCY_PRIORITY"].fillna(0, inplace=True)

C:\Users\ianta\AppData\Local\Temp\ipykernel_9928\1450970830.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Delay_Reason"].fillna(0, inplace=True)
C:\Users\ianta\AppData\Local\Temp\ipykernel_9928\1450970830.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

### Error Correction (By Eyeballing)

- Date for row OPERATION_ID == 582117 should be 2010-04-19.

In [3731]:
planned_cols = [
        "PLANNED_PATIENT_CALL_TIME",
        "PLANNED_PATIENT_FETCH_TIME",
        "PLANNED_RECEPTION_IN_TIME",
        "PLANNED_ENTER_OR_TIME",
        "PLANNED_SURGERY_PREP_TIME",
        "PLANNED_ANAESTHESIA_INDUCTION",
        "PLANNED_KNIFE_TO_SKIN_TIME",
        "PLANNED_SKIN_CLOSURE",
        "PLANNED_PATIENT_REVERSAL_TIME",
        "PLANNED_EXIT_OR_TIME",
        "PLANNED_OR_CLEANUP_TIME",
        "PLANNED_EXIT_RECOVERY_TIME",        
    ]

actual_cols = [
        "PATIENT_CALL_TIME",
        "PATIENT_FETCH_TIME",
        "ACTUAL_RECEPTION_IN_TIME",
        "ACTUAL_ENTER_OR_TIME",
        "ACTUAL_SURGERY_PREP_TIME",
        "ACTUAL_ANAESTHESIA_INDUCTION",
        "ACTUAL_KNIFE_TO_SKIN_TIME",
        "ACTUAL_SKIN_CLOSURE",
        "ACTUAL_PATIENT_REVERSAL_TIME",
        "ACTUAL_EXIT_OR_TIME",
        "ACTUAL_OR_CLEANUP_TIME",
        "ACTUAL_EXIT_RECOVERY_TIME",        
    ]

# target row
target_id = 582117
fix_date = pd.Timestamp("2010-04-19")

mask = data["OPERATION_ID"] == target_id

def force_date(val, base_date):
    if pd.isna(val):
        return val
    # If it's already a Timestamp
    if isinstance(val, pd.Timestamp):
        return pd.Timestamp.combine(base_date, val.time())
    # If it's a datetime.time
    if isinstance(val, datetime.time):
        return pd.Timestamp.combine(base_date, val)
    # Try to parse strings or other objects
    try:
        parsed = pd.to_datetime(val, errors="coerce")
        if pd.isna(parsed):
            return val
        return pd.Timestamp.combine(base_date, parsed.time())
    except Exception:
        return val

for col in planned_cols:
    if col in data.columns:
        data.loc[mask, col] = data.loc[mask, col].apply(lambda v: force_date(v, fix_date))

for col in actual_cols:
    if col in data.columns:
        data.loc[mask, col] = data.loc[mask, col].apply(lambda v: force_date(v, fix_date))        

data.iloc[:, 7:31]

,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,PLANNED_ENTER_OR_TIME,PLANNED_ANAESTHESIA_INDUCTION,PLANNED_SURGERY_PREP_TIME,PLANNED_KNIFE_TO_SKIN_TIME,PLANNED_SKIN_CLOSURE,PLANNED_PATIENT_REVERSAL_TIME,PLANNED_EXIT_OR_TIME,...,ACTUAL_RECEPTION_IN_TIME,ACTUAL_ENTER_OR_TIME,ACTUAL_ANAESTHESIA_INDUCTION,ACTUAL_SURGERY_PREP_TIME,ACTUAL_KNIFE_TO_SKIN_TIME,ACTUAL_SKIN_CLOSURE,ACTUAL_PATIENT_REVERSAL_TIME,ACTUAL_EXIT_OR_TIME,ACTUAL_EXIT_RECOVERY_TIME,ACTUAL_OR_CLEANUP_TIME
0,09:50:00,2011-04-19 09:50:00,09:50:00,09:50:00,2011-04-19 10:20:00,2011-04-19 10:20:00,10:20:00,17:10:00,2011-04-19 17:10:00,17:25:00,...,10:41:00,11:04:00,2011-04-19 11:06:00,2011-04-19 11:17:00,11:44:00,17:31:00,2011-04-19 17:43:00,17:46:00,18:48:00,NaT
1,10:40:00,2011-04-19 10:40:00,10:40:00,10:40:00,2011-04-19 10:50:00,2011-04-19 10:50:00,10:50:00,11:20:00,2011-04-19 11:20:00,11:25:00,...,10:17:00,11:01:00,2011-04-19 11:03:00,2011-04-19 11:08:00,11:10:00,11:20:00,2011-04-19 11:32:00,11:32:00,12:48:00,2011-04-19 11:33:00
2,10:55:00,2011-04-19 10:55:00,10:55:00,10:55:00,NaT,NaT,11:10:00,11:55:00,NaT,11:55:00,...,10:39:00,10:58:00,NaT,NaT,11:02:00,11:21:00,NaT,11:24:00,11:44:00,2011-04-19 13:35:00
3,10:50:00,2011-04-19 10:50:00,10:50:00,10:50:00,2011-04-19 11:10:00,2011-04-19 11:10:00,11:10:00,13:40:00,2011-04-19 13:40:00,13:50:00,...,10:36:00,10:58:00,2011-04-19 10:59:00,2011-04-19 11:09:00,11:17:00,13:09:00,2011-04-19 13:09:00,13:27:00,15:10:00,2011-04-19 13:27:00
4,10:30:00,2011-04-19 10:30:00,10:30:00,10:30:00,2011-04-19 10:50:00,2011-04-19 10:50:00,10:50:00,12:20:00,2011-04-19 12:20:00,12:30:00,...,10:58:00,10:58:00,2011-04-19 11:01:00,2011-04-19 11:23:00,11:24:00,13:17:00,2011-04-19 13:28:00,13:34:00,18:41:00,2011-04-19 13:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276858,09:25:00,NaT,09:25:00,09:25:00,2019-06-20 09:35:00,NaT,09:35:00,09:48:00,2019-06-20 09:48:00,09:50:00,...,08:29:00,09:44:00,2019-06-20 09:49:00,NaT,09:50:00,10:05:00,2019-06-20 10:05:00,10:07:00,11:18:00,2019-06-20 10:37:00
276859,12:45:00,2019-06-20 12:45:00,12:45:00,12:45:00,2019-06-20 13:15:00,2019-06-20 13:15:00,13:15:00,17:20:00,2019-06-20 17:20:00,17:30:00,...,09:13:00,09:36:00,2019-06-20 09:38:00,2019-06-20 09:50:00,10:19:00,13:23:00,2019-06-20 13:23:00,13:51:00,16:14:00,2019-06-20 14:01:00
276860,08:30:00,NaT,08:30:00,08:30:00,2019-06-20 08:40:00,NaT,08:40:00,08:48:00,2019-06-20 08:48:00,08:50:00,...,08:32:00,09:20:00,2019-06-20 09:27:00,NaT,09:30:00,09:50:00,2019-06-20 09:50:00,09:52:00,10:16:00,2019-06-20 09:53:00
276861,09:20:00,2019-06-20 09:20:00,09:20:00,09:20:00,2019-06-20 09:35:00,2019-06-20 09:35:00,09:35:00,09:55:00,2019-06-20 09:55:00,10:00:00,...,09:20:00,09:20:00,2019-06-20 09:34:00,2019-06-20 09:40:00,09:46:00,10:03:00,2019-06-20 10:04:00,10:08:00,10:40:00,2019-06-20 10:10:00


### Date Handling and Imputation Rules

This section handles messy date/time data by enforcing **consistent start dates** and applying **domain-specific sync rules**.

---

#### 1. Detecting Date vs. Time-only Strings
- `_looks_like_date_string(s)` → checks if a string contains a date-like pattern (`YYYY-MM-DD`, `DD/MM/YYYY`, etc.).  
- `_is_time_only_string(s)` → checks if a string looks like a time-only entry (`08:15`, `8:15:00 AM`, etc.).

This distinction allows us to avoid misinterpreting time-only values as full datetimes.

---

#### 2. Determining Constant Start Dates
- `find_start_date_from_row(row, cols)` scans a list of columns in a row and finds the **first valid date**.  
  - A valid date is a `Timestamp` with `year > 1900` or a parseable date string.  
  - Returns the **normalized date** (time set to 00:00:00).  

- `attach_constant_dates(row, planned_cols, actual_cols)`:
  - Finds one **planned_start** and one **actual_start** per row.
  - If `actual_start` is missing but `planned_start` exists, use the planned date as fallback.
  - For any time-only strings, attach the corresponding start date to construct a full `Timestamp`.
  - Full datetime values are preserved as-is.

---

#### 2b. Date Sanitisation 
We discovered that some of the input data, while they had dates, had **corrupted or incorrectly manipulated dates**.  
To ensure all downstream imputations are built on reliable timelines, we enforce a **sanity check**:

1. **Valid date range**  
   - Earliest allowed date: **2016-12-31**  
   - Latest allowed date: **2022-02-25**

2. **Correction procedure**  
   - For each row, check if `planned_start` and `actual_start` fall within the valid range.  
   - If either date is outside this range, attempt to backfill the correct value from  
     `oots-cleaned-unlocked.xlsx` using `OPERATION_ID`.  
   - If no match is found in the validation file, replace the invalid date with `NaT`.

3. **Guarantees after cleaning**  
   - Every `planned_start` and `actual_start` is either:
     - Within the valid range, or  
     - Backfilled from the validation dataset, or  
     - Explicitly marked as `NaT` if no trusted source is available.  

This step ensures that **all subsequent imputations** (e.g., filling missing times)  
operate only on dates within the trusted window.

---

#### 3. Row-wise Imputation Rules
- `impute_with_rules(row, planned_cols, actual_cols)`:
  1. **Attach constant start dates** using `attach_constant_dates`.
  2. **Sync critical columns**:
     - `PLANNED_PATIENT_CALL_TIME` ↔ `PLANNED_PATIENT_FETCH_TIME`  
     - `PLANNED_OR_CLEANUP_TIME` ↔ `PLANNED_EXIT_OR_TIME`  
     Preference is given to whichever value exists.
  3. **Enforce ordering constraints**:
     - Knife-to-skin ≤ Skin closure ≤ Patient reversal ≤ Exit OR ≤ Exit recovery ≤ OR cleanup
     - If any step goes backwards, adjust forward.
  4. **Fill missing anchor values**:
     - If missing, `PLANNED_ANAESTHESIA_INDUCTION` and `PLANNED_SURGERY_PREP_TIME` are set to `PLANNED_KNIFE_TO_SKIN_TIME`.
     - If missing, `PLANNED_PATIENT_REVERSAL_TIME` is set to `PLANNED_SKIN_CLOSURE`.

In [3732]:
MIN_DATE = pd.Timestamp("2016-12-31")
MAX_DATE = pd.Timestamp("2022-02-25")

warnings = []

_time_only_re = re.compile(r'^\s*\d{1,2}:\d{2}(:\d{2})?\s*(?:[AaPp][Mm])?\s*$')
_date_like_re = re.compile(r'\d{4}[-/]\d{1,2}[-/]\d{1,2}|\d{1,2}[-/]\d{1,2}[-/]\d{2,4}')

def _looks_like_date_string(s: str) -> bool:
    """Rudimentary check whether a string contains an explicit date."""
    if not isinstance(s, str):
        return False
    s = s.strip()
    return bool(_date_like_re.search(s))

def _is_time_only_string(s: str) -> bool:
    """True if string appears to contain time only (e.g. '08:15' or '8:15:00 AM')."""
    if not isinstance(s, str):
        return False
    return bool(_time_only_re.match(s.strip()))

def find_start_date_from_row(row, cols):
    """
    Scan cols in order and return the first discovered 'date' (normalized).
    We consider a value to contain a date if:
      - it's a pandas Timestamp / datetime with a year > 1900
      - or the original string contains a date-like pattern and parses to a Timestamp with sensible year
    """
    for col in cols:
        if col not in row.index:
            continue
        val = row[col]
        if pd.isna(val):
            continue

        # Already a Timestamp / datetime
        if isinstance(val, pd.Timestamp):
            if val.year > 1900:  # treat as containing a real date
                return val.normalize()
            else:
                # likely a parsed time-only; skip
                continue
        if isinstance(val, datetime.datetime):
            if val.year > 1900:
                return pd.Timestamp(val).normalize()

        # If it's a string, check if it looks like a date first
        try:
            s = str(val).strip()
        except Exception:
            continue

        if _looks_like_date_string(s):
            parsed = pd.to_datetime(s, errors="coerce", dayfirst=False)
            if not pd.isna(parsed) and parsed.year > 1900:
                return parsed.normalize()

        # If it wasn't date-like, skip (likely time-only)
    return None


def attach_constant_dates(row, planned_cols, actual_cols):
    """
    For a given row:
     - find planned_start = first planned col that contains a date
     - find actual_start  = first actual col that contains a date
     - verify both are within MIN_DATE .. MAX_DATE; if not, try to retrieve from validation_date by OPERATION_ID
     - for each planned col: if its value is time-only, attach planned_start
     - for each actual col: if its value is time-only, attach actual_start
    Does NOT override full datetimes.
    """
    # Find constants (scan entire list until first real-date found)
    planned_start = find_start_date_from_row(row, planned_cols)
    actual_start = find_start_date_from_row(row, actual_cols)

    # --- SANITY: if either start is missing or out of acceptable range, try to pull from validation sheet ---
    def _in_range(ts):
        return isinstance(ts, pd.Timestamp) and (MIN_DATE <= ts <= MAX_DATE)

    # helper to attempt retrieval from validation_data using OPERATION_ID
    def _try_validation_lookup(opid, cols):
        if opid is None:
            return None
        # attempt direct match first; fall back to string match if needed
        matches = pd.DataFrame()
        try:
            matches = validation_data.loc[validation_data["OPERATION_ID"] == opid]
        except Exception:
            try:
                matches = validation_data.loc[validation_data["OPERATION_ID"].astype(str) == str(opid)]
            except Exception:
                matches = pd.DataFrame()

        if matches is not None and not matches.empty:
            val_row = matches.iloc[0]
            return find_start_date_from_row(val_row, cols)
        return None

    opid = row.get("OPERATION_ID", None)

    # planned_start: if missing or out-of-range, attempt fallback from validation sheet
    if not _in_range(planned_start):
        alt_planned = _try_validation_lookup(opid, planned_cols)
        if _in_range(alt_planned):
            planned_start = alt_planned
            warnings.append(f"OPERATION_ID={opid}: planned_start replaced from validation file ({planned_start.date()}).")
        else:
            warnings.append(f"OPERATION_ID={opid}: planned_start {planned_start} out of range or missing; no valid replacement found in validation file.")
            planned_start = None

    # actual_start: if missing or out-of-range, attempt fallback from validation sheet
    if not _in_range(actual_start):
        alt_actual = _try_validation_lookup(opid, actual_cols)
        if _in_range(alt_actual):
            actual_start = alt_actual
            warnings.append(f"OPERATION_ID={opid}: actual_start replaced from validation file ({actual_start.date()}).")
        else:
            # if no actual start in validation, we will keep None for now and allow later fallback to planned_start
            warnings.append(f"OPERATION_ID={opid}: actual_start {actual_start} out of range or missing; no valid replacement found in validation file.")
            actual_start = None

    # --- NEW: fallback (if actual still missing, use planned_start) ---
    if actual_start is None and planned_start is not None:
        actual_start = planned_start

    # Helper to combine time-only string with a start date
    def _combine_time_with_date(s, base_date):
        # parse the time string into a datetime (may get today's date, we only use .time())
        parsed = pd.to_datetime(s, errors="coerce")
        if pd.isna(parsed) or base_date is None:
            return None
        return pd.Timestamp.combine(base_date, parsed.time())

    # Fill planned cols
    for col in planned_cols:
        if col not in row.index:
            continue
        val = row[col]
        if pd.isna(val):
            continue

        # if it's already a Timestamp or datetime
        if isinstance(val, (pd.Timestamp, datetime.datetime)):
            ts = pd.Timestamp(val)
            if MIN_DATE <= ts <= MAX_DATE:
                row[col] = ts
                continue
            else:
                # invalid → try to replace with same time on planned_start
                if planned_start is not None:
                    row[col] = pd.Timestamp.combine(planned_start, ts.time())
                else:
                    row[col] = pd.NaT
            continue

        # otherwise, treat it as string
        s = str(val).strip()
        if _is_time_only_string(s):
            if planned_start is not None:
                combined = _combine_time_with_date(s, planned_start)
                if combined is not None:
                    row[col] = combined
        else:
            parsed = pd.to_datetime(s, errors="coerce", dayfirst=False)
            if not pd.isna(parsed):
                if MIN_DATE <= parsed <= MAX_DATE:
                    row[col] = parsed
                elif planned_start is not None:
                    row[col] = pd.Timestamp.combine(planned_start, parsed.time())
                else:
                    row[col] = pd.NaT

    # Fill actual cols (same logic)
    for col in actual_cols:
        if col not in row.index:
            continue
        val = row[col]
        if pd.isna(val):
            continue

        if isinstance(val, (pd.Timestamp, datetime.datetime)):
            ts = pd.Timestamp(val)
            if MIN_DATE <= ts <= MAX_DATE:
                row[col] = ts
                continue
            else:
                if actual_start is not None:
                    row[col] = pd.Timestamp.combine(actual_start, ts.time())
                else:
                    row[col] = pd.NaT
            continue

        s = str(val).strip()
        if _is_time_only_string(s):
            if actual_start is not None:
                combined = _combine_time_with_date(s, actual_start)
                if combined is not None:
                    row[col] = combined
        else:
            parsed = pd.to_datetime(s, errors="coerce", dayfirst=False)
            if not pd.isna(parsed):
                if MIN_DATE <= parsed <= MAX_DATE:
                    row[col] = parsed
                elif actual_start is not None:
                    row[col] = pd.Timestamp.combine(actual_start, parsed.time())
                else:
                    row[col] = pd.NaT


    return row


def impute_with_rules(row, planned_cols, actual_cols):
    """
    Step 1: Attach constant planned/actual start dates
    Step 2: Apply sync rules (with bias toward existing non-null values)
    Step 3: Fill missing anaesthesia/prep times from knife-to-skin
    """
    # --- Step 1: attach constant dates ---
    row = attach_constant_dates(row, planned_cols, actual_cols)

    # --- Step 2: enforce logical sync rules ---
    def sync_cols(col_a, col_b, prefer="a"):
        """Sync two columns with preference if one is missing."""
        a, b = row.get(col_a, pd.NaT), row.get(col_b, pd.NaT)
        if pd.isna(a) and pd.notna(b):
            row[col_a] = b
        elif pd.isna(b) and pd.notna(a):
            row[col_b] = a
        elif pd.notna(a) and pd.notna(b):
            if prefer == "a":
                row[col_b] = a
            else:
                row[col_a] = b

    # Rule 1: PLANNED_PATIENT_CALL_TIME == PLANNED_PATIENT_FETCH_TIME
    sync_cols("PLANNED_PATIENT_CALL_TIME", "PLANNED_PATIENT_FETCH_TIME", prefer="fetch")

    # Rule 2: PLANNED_OR_CLEANUP_TIME == PLANNED_EXIT_OR_TIME
    sync_cols("PLANNED_OR_CLEANUP_TIME", "PLANNED_EXIT_OR_TIME", prefer="exit")

    # Rule 3: Ensure ordering constraints (only if both present)
    def enforce_order(before, after):
        if before in row.index and after in row.index:
            if pd.notna(row[before]) and pd.notna(row[after]):
                try:
                    if row[after] < row[before]:
                        row[after] = row[before]
                except Exception:
                    pass

    enforce_order("PLANNED_KNIFE_TO_SKIN_TIME", "PLANNED_SKIN_CLOSURE")
    enforce_order("PLANNED_SKIN_CLOSURE", "PLANNED_PATIENT_REVERSAL_TIME")
    enforce_order("PLANNED_PATIENT_REVERSAL_TIME", "PLANNED_EXIT_OR_TIME")
    enforce_order("PLANNED_EXIT_OR_TIME", "PLANNED_EXIT_RECOVERY_TIME")
    enforce_order("PLANNED_EXIT_RECOVERY_TIME", "PLANNED_OR_CLEANUP_TIME")

    # --- Step 3: fill missing times from anchors ---
    knife = row.get("PLANNED_KNIFE_TO_SKIN_TIME", pd.NaT)
    closure = row.get("PLANNED_SKIN_CLOSURE", pd.NaT)

    if pd.notna(knife):
        if "PLANNED_ANAESTHESIA_INDUCTION" in row.index and pd.isna(row["PLANNED_ANAESTHESIA_INDUCTION"]):
            row["PLANNED_ANAESTHESIA_INDUCTION"] = knife
        if "PLANNED_SURGERY_PREP_TIME" in row.index and pd.isna(row["PLANNED_SURGERY_PREP_TIME"]):
            row["PLANNED_SURGERY_PREP_TIME"] = knife

    if pd.notna(closure):
        if "PLANNED_PATIENT_REVERSAL_TIME" in row.index and pd.isna(row["PLANNED_PATIENT_REVERSAL_TIME"]):
            row["PLANNED_PATIENT_REVERSAL_TIME"] = closure

    return row

# Apply row-wise
data = data.apply(lambda r: impute_with_rules(r, planned_cols, actual_cols), axis=1)
data.iloc[:, 7:31]

,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,PLANNED_ENTER_OR_TIME,PLANNED_ANAESTHESIA_INDUCTION,PLANNED_SURGERY_PREP_TIME,PLANNED_KNIFE_TO_SKIN_TIME,PLANNED_SKIN_CLOSURE,PLANNED_PATIENT_REVERSAL_TIME,PLANNED_EXIT_OR_TIME,...,ACTUAL_RECEPTION_IN_TIME,ACTUAL_ENTER_OR_TIME,ACTUAL_ANAESTHESIA_INDUCTION,ACTUAL_SURGERY_PREP_TIME,ACTUAL_KNIFE_TO_SKIN_TIME,ACTUAL_SKIN_CLOSURE,ACTUAL_PATIENT_REVERSAL_TIME,ACTUAL_EXIT_OR_TIME,ACTUAL_EXIT_RECOVERY_TIME,ACTUAL_OR_CLEANUP_TIME
0,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 10:20:00,2019-04-11 10:20:00,2019-04-11 10:20:00,2019-04-11 17:10:00,2019-04-11 17:10:00,2019-04-11 17:25:00,...,2019-04-11 10:41:00,2019-04-11 11:04:00,2019-04-11 11:06:00,2019-04-11 11:17:00,2019-04-11 11:44:00,2019-04-11 17:31:00,2019-04-11 17:43:00,2019-04-11 17:46:00,2019-04-11 18:48:00,NaT
1,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 11:20:00,2019-04-11 11:20:00,2019-04-11 11:25:00,...,2019-04-11 10:17:00,2019-04-11 11:01:00,2019-04-11 11:03:00,2019-04-11 11:08:00,2019-04-11 11:10:00,2019-04-11 11:20:00,2019-04-11 11:32:00,2019-04-11 11:32:00,2019-04-11 12:48:00,2019-04-11 11:33:00
2,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 11:10:00,2019-04-11 11:10:00,2019-04-11 11:10:00,2019-04-11 11:55:00,2019-04-11 11:55:00,2019-04-11 11:55:00,...,2019-04-11 10:39:00,2019-04-11 10:58:00,NaT,NaT,2019-04-11 11:02:00,2019-04-11 11:21:00,NaT,2019-04-11 11:24:00,2019-04-11 11:44:00,2019-04-11 13:35:00
3,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 11:10:00,2019-04-11 11:10:00,2019-04-11 11:10:00,2019-04-11 13:40:00,2019-04-11 13:40:00,2019-04-11 13:50:00,...,2019-04-11 10:36:00,2019-04-11 10:58:00,2019-04-11 10:59:00,2019-04-11 11:09:00,2019-04-11 11:17:00,2019-04-11 13:09:00,2019-04-11 13:09:00,2019-04-11 13:27:00,2019-04-11 15:10:00,2019-04-11 13:27:00
4,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 12:20:00,2019-04-11 12:20:00,2019-04-11 12:30:00,...,2019-04-11 10:58:00,2019-04-11 10:58:00,2019-04-11 11:01:00,2019-04-11 11:23:00,2019-04-11 11:24:00,2019-04-11 13:17:00,2019-04-11 13:28:00,2019-04-11 13:34:00,2019-04-11 18:41:00,2019-04-11 13:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276858,2019-06-20 09:25:00,2019-06-20 09:25:00,2019-06-20 09:25:00,2019-06-20 09:25:00,2019-06-20 09:35:00,2019-06-20 09:35:00,2019-06-20 09:35:00,2019-06-20 09:48:00,2019-06-20 09:48:00,2019-06-20 09:50:00,...,2019-06-20 08:29:00,2019-06-20 09:44:00,2019-06-20 09:49:00,NaT,2019-06-20 09:50:00,2019-06-20 10:05:00,2019-06-20 10:05:00,2019-06-20 10:07:00,2019-06-20 11:18:00,2019-06-20 10:37:00
276859,2019-06-20 12:45:00,2019-06-20 12:45:00,2019-06-20 12:45:00,2019-06-20 12:45:00,2019-06-20 13:15:00,2019-06-20 13:15:00,2019-06-20 13:15:00,2019-06-20 17:20:00,2019-06-20 17:20:00,2019-06-20 17:30:00,...,2019-06-20 09:13:00,2019-06-20 09:36:00,2019-06-20 09:38:00,2019-06-20 09:50:00,2019-06-20 10:19:00,2019-06-20 13:23:00,2019-06-20 13:23:00,2019-06-20 13:51:00,2019-06-20 16:14:00,2019-06-20 14:01:00
276860,2019-06-20 08:30:00,2019-06-20 08:30:00,2019-06-20 08:30:00,2019-06-20 08:30:00,2019-06-20 08:40:00,2019-06-20 08:40:00,2019-06-20 08:40:00,2019-06-20 08:48:00,2019-06-20 08:48:00,2019-06-20 08:50:00,...,2019-06-20 08:32:00,2019-06-20 09:20:00,2019-06-20 09:27:00,NaT,2019-06-20 09:30:00,2019-06-20 09:50:00,2019-06-20 09:50:00,2019-06-20 09:52:00,2019-06-20 10:16:00,2019-06-20 09:53:00
276861,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:35:00,2019-06-20 09:35:00,2019-06-20 09:35:00,2019-06-20 09:55:00,2019-06-20 09:55:00,2019-06-20 10:00:00,...,2019-06-20 09:20:00,2019-06-2

Date processing for ACTUAL columns

In [3733]:
def impute_patient_times(row):
    fetch_col = "PATIENT_FETCH_TIME"
    call_col = "PATIENT_CALL_TIME"
    reception_col = "ACTUAL_RECEPTION_IN_TIME"

    # Helper: coerce any value to Timestamp
    def _to_ts(val):
        if pd.isna(val):
            return None
        if isinstance(val, pd.Timestamp):
            return val
        try:
            return pd.to_datetime(val, errors="coerce")
        except Exception:
            return None

    # --- Step 1: if CALL is empty, copy RECEPTION ---
    call_time = _to_ts(row.get(call_col, pd.NaT))
    reception_time = _to_ts(row.get(reception_col, pd.NaT))

    if call_time is None and reception_time is not None:
        row[call_col] = reception_time
        call_time = reception_time

    # --- Step 2: if FETCH is empty, fill it ---
    fetch_time = _to_ts(row.get(fetch_col, pd.NaT))
    if fetch_time is None:
        if call_time is not None and reception_time is not None:
            midpoint = call_time + (reception_time - call_time) / 2
            row[fetch_col] = midpoint.floor("min")  # round down
        elif call_time is not None:
            row[fetch_col] = call_time.floor("min")
        elif reception_time is not None:
            row[fetch_col] = reception_time.floor("min")

    return row

data = data.apply(impute_patient_times, axis=1)

data.iloc[:, 19:31]

,PATIENT_CALL_TIME,PATIENT_FETCH_TIME,ACTUAL_RECEPTION_IN_TIME,ACTUAL_ENTER_OR_TIME,ACTUAL_ANAESTHESIA_INDUCTION,ACTUAL_SURGERY_PREP_TIME,ACTUAL_KNIFE_TO_SKIN_TIME,ACTUAL_SKIN_CLOSURE,ACTUAL_PATIENT_REVERSAL_TIME,ACTUAL_EXIT_OR_TIME,ACTUAL_EXIT_RECOVERY_TIME,ACTUAL_OR_CLEANUP_TIME
0,2019-04-11 10:15:00,2019-04-11 10:26:00,2019-04-11 10:41:00,2019-04-11 11:04:00,2019-04-11 11:06:00,2019-04-11 11:17:00,2019-04-11 11:44:00,2019-04-11 17:31:00,2019-04-11 17:43:00,2019-04-11 17:46:00,2019-04-11 18:48:00,NaT
1,2019-04-11 09:49:00,2019-04-11 09:55:00,2019-04-11 10:17:00,2019-04-11 11:01:00,2019-04-11 11:03:00,2019-04-11 11:08:00,2019-04-11 11:10:00,2019-04-11 11:20:00,2019-04-11 11:32:00,2019-04-11 11:32:00,2019-04-11 12:48:00,2019-04-11 11:33:00
2,2019-04-11 10:15:00,2019-04-11 10:15:00,2019-04-11 10:39:00,2019-04-11 10:58:00,NaT,NaT,2019-04-11 11:02:00,2019-04-11 11:21:00,NaT,2019-04-11 11:24:00,2019-04-11 11:44:00,2019-04-11 13:35:00
3,2019-04-11 10:15:00,2019-04-11 10:18:00,2019-04-11 10:36:00,2019-04-11 10:58:00,2019-04-11 10:59:00,2019-04-11 11:09:00,2019-04-11 11:17:00,2019-04-11 13:09:00,2019-04-11 13:09:00,2019-04-11 13:27:00,2019-04-11 15:10:00,2019-04-11 13:27:00
4,2019-04-11 09:45:00,2019-04-11 10:44:00,2019-04-11 10:58:00,2019-04-11 10:58:00,2019-04-11 11:01:00,2019-04-11 11:23:00,2019-04-11 11:24:00,2019-04-11 13:17:00,2019-04-11 13:28:00,2019-04-11 13:34:00,2019-04-11 18:41:00,2019-04-11 13:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...
276858,2019-06-20 08:24:00,2019-06-20 08:26:00,2019-06-20 08:29:00,2019-06-20 09:44:00,2019-06-20 09:49:00,NaT,2019-06-20 09:50:00,2019-06-20 10:05:00,2019-06-20 10:05:00,2019-06-20 10:07:00,2019-06-20 11:18:00,2019-06-20 10:37:00
276859,2019-06-20 08:51:00,2019-06-20 09:04:00,2019-06-20 09:13:00,2019-06-20 09:36:00,2019-06-20 09:38:00,2019-06-20 09:50:00,2019-06-20 10:19:00,2019-06-20 13:23:00,2019-06-20 13:23:00,2019-06-20 13:51:00,2019-06-20 16:14:00,2019-06-20 14:01:00
276860,2019-06-20 08:11:00,2019-06-20 08:21:00,2019-06-20 08:32:00,2019-06-20 09:20:00,2019-06-20 09:27:00,NaT,2019-06-20 09:30:00,2019-06-20 09:50:00,2019-06-20 09:50:00,2019-06-20 09:52:00,2019-06-20 10:16:00,2019-06-20 09:53:00
276861,2019-06-20 07:44:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:34:00,2019-06-20 09:40:00,2019-06-20 09:46:00,2019-06-20 10:03:00,2019-06-20 10:04:00,2019-06-20 10:08:00,2019-06-20 10:40:00,2019-06-20 10:10:00


### Imputing Missing Induction, Prep, and Reversal Times

For some rows, `ACTUAL_ANAESTHESIA_INDUCTION`, `ACTUAL_SURGERY_PREP_TIME`,   
`ACTUAL_PATIENT_REVERSAL_TIME`, and `ACTUAL_OR_CLEANUP_TIME` are missing.  
To fill these values in a consistent and data-driven way, we treat the **OR workflow as a timeline**.

---

#### Case A: Induction & Prep together (baseline method)
- `ACTUAL_ENTER_OR_TIME` → **0% mark**  
- `ACTUAL_KNIFE_TO_SKIN_TIME` → **100% mark**  

For rows where both induction and prep times are available, we compute their relative positions:
- **Induction mark** = (Induction − Enter OR) ÷ (Knife-to-skin − Enter OR)  
- **Prep mark** = (Prep − Enter OR) ÷ (Knife-to-skin − Enter OR)  

We then take the **average mark** across all valid rows.  
For rows with missing values:
- `ACTUAL_ANAESTHESIA_INDUCTION` is backfilled as  
  `Enter OR + (Knife-to-skin − Enter OR) × <avg induction mark>`, rounded to the nearest minute  
- `ACTUAL_SURGERY_PREP_TIME` is backfilled as  
  `Enter OR + (Knife-to-skin − Enter OR) × <avg prep mark>`, rounded to the nearest minute  

---

#### Case B: Prep missing, but induction & knife available
For rows with induction and knife-to-skin times but missing prep:  
- Compute average **prep-from-induction mark** = (Prep − Induction) ÷ (Knife − Induction)  
- Backfill missing prep as  
  `Induction + (Knife − Induction) × <avg prep-from-induction mark>`

---

#### Case C: Induction missing, but enter & prep available
For rows with enter and prep but missing induction:  
- Compute average **induction-from-enter mark** = (Induction − Enter OR) ÷ (Prep − Enter OR)  
- Backfill missing induction as  
  `Enter OR + (Prep − Enter OR) × <avg induction-from-enter mark>`

---

#### Case D: Reversal missing, but closure & exit available
For rows with closure and exit but missing reversal:  
- `ACTUAL_SKIN_CLOSURE` → **0% mark**  
- `ACTUAL_EXIT_OR_TIME` → **100% mark**  
- Compute average **reversal mark** = (Reversal − Closure) ÷ (Exit − Closure)  
- Backfill missing reversal as  
  `Closure + (Exit − Closure) × <avg reversal mark>`, rounded to the nearest minute  

---

#### Case E: Cleanup missing, but exit available
Unlike induction, prep, and reversal, cleanup is best modeled as a **fixed offset** after exit.  
- Compute average **cleanup offset** = (Cleanup − Exit) across rows with both values.  
- Backfill missing cleanup as  
  `Exit + <avg cleanup offset>`  
Rounded to the nearest minute.

---

This imputation strategy ensures that filled values preserve the natural ordering of OR events, are grounded in real observed distributions, and remain realistic within the surgical timeline.

In [3734]:
def compute_marks(data):
    marks = {}

    # Case A: induction & prep relative to enter/knife
    mask = (
        data["ACTUAL_ENTER_OR_TIME"].notna()
        & data["ACTUAL_ANAESTHESIA_INDUCTION"].notna()
        & data["ACTUAL_SURGERY_PREP_TIME"].notna()
        & data["ACTUAL_KNIFE_TO_SKIN_TIME"].notna()
    )
    clean = data.loc[mask].copy()
    clean = clean[
        (clean["ACTUAL_ENTER_OR_TIME"] <= clean["ACTUAL_ANAESTHESIA_INDUCTION"])
        & (clean["ACTUAL_ANAESTHESIA_INDUCTION"] <= clean["ACTUAL_SURGERY_PREP_TIME"])
        & (clean["ACTUAL_SURGERY_PREP_TIME"] <= clean["ACTUAL_KNIFE_TO_SKIN_TIME"])
    ]
    if not clean.empty:
        total = (clean["ACTUAL_KNIFE_TO_SKIN_TIME"] - clean["ACTUAL_ENTER_OR_TIME"]).dt.total_seconds()
        marks["induction"] = ((clean["ACTUAL_ANAESTHESIA_INDUCTION"] - clean["ACTUAL_ENTER_OR_TIME"]).dt.total_seconds() / total).mean(skipna=True)
        marks["prep"] = ((clean["ACTUAL_SURGERY_PREP_TIME"] - clean["ACTUAL_ENTER_OR_TIME"]).dt.total_seconds() / total).mean(skipna=True)

    # Case B: prep relative to induction/knife
    mask = (
        data["ACTUAL_ANAESTHESIA_INDUCTION"].notna()
        & data["ACTUAL_SURGERY_PREP_TIME"].notna()
        & data["ACTUAL_KNIFE_TO_SKIN_TIME"].notna()
    )
    clean = data.loc[mask].copy()
    clean = clean[
        (clean["ACTUAL_ANAESTHESIA_INDUCTION"] <= clean["ACTUAL_SURGERY_PREP_TIME"])
        & (clean["ACTUAL_SURGERY_PREP_TIME"] <= clean["ACTUAL_KNIFE_TO_SKIN_TIME"])
    ]
    if not clean.empty:
        total = (clean["ACTUAL_KNIFE_TO_SKIN_TIME"] - clean["ACTUAL_ANAESTHESIA_INDUCTION"]).dt.total_seconds()
        marks["prep_from_induction"] = ((clean["ACTUAL_SURGERY_PREP_TIME"] - clean["ACTUAL_ANAESTHESIA_INDUCTION"]).dt.total_seconds() / total).mean(skipna=True)

    # Case C: induction relative to enter/prep
    mask = (
        data["ACTUAL_ENTER_OR_TIME"].notna()
        & data["ACTUAL_ANAESTHESIA_INDUCTION"].notna()
        & data["ACTUAL_SURGERY_PREP_TIME"].notna()
    )
    clean = data.loc[mask].copy()
    clean = clean[
        (clean["ACTUAL_ENTER_OR_TIME"] <= clean["ACTUAL_ANAESTHESIA_INDUCTION"])
        & (clean["ACTUAL_ANAESTHESIA_INDUCTION"] <= clean["ACTUAL_SURGERY_PREP_TIME"])
    ]
    if not clean.empty:
        total = (clean["ACTUAL_SURGERY_PREP_TIME"] - clean["ACTUAL_ENTER_OR_TIME"]).dt.total_seconds()
        marks["induction_from_enter"] = ((clean["ACTUAL_ANAESTHESIA_INDUCTION"] - clean["ACTUAL_ENTER_OR_TIME"]).dt.total_seconds() / total).mean(skipna=True)

    # Case D: reversal relative to closure/exit
    mask = (
        data["ACTUAL_SKIN_CLOSURE"].notna()
        & data["ACTUAL_PATIENT_REVERSAL_TIME"].notna()
        & data["ACTUAL_EXIT_OR_TIME"].notna()
    )
    clean = data.loc[mask].copy()
    clean = clean[
        (clean["ACTUAL_SKIN_CLOSURE"] <= clean["ACTUAL_PATIENT_REVERSAL_TIME"])
        & (clean["ACTUAL_PATIENT_REVERSAL_TIME"] <= clean["ACTUAL_EXIT_OR_TIME"])
    ]
    if not clean.empty:
        total = (clean["ACTUAL_EXIT_OR_TIME"] - clean["ACTUAL_SKIN_CLOSURE"]).dt.total_seconds()
        marks["reversal"] = ((clean["ACTUAL_PATIENT_REVERSAL_TIME"] - clean["ACTUAL_SKIN_CLOSURE"]).dt.total_seconds() / total).mean(skipna=True)

    # Case E: cleanup offset from exit
    mask = (
        data["ACTUAL_EXIT_OR_TIME"].notna()
        & data["ACTUAL_OR_CLEANUP_TIME"].notna()
    )
    clean = data.loc[mask].copy()

    # Keep only realistic differences (0 to 12 hours after exit)
    valid = (clean["ACTUAL_OR_CLEANUP_TIME"] >= clean["ACTUAL_EXIT_OR_TIME"]) & (
        (clean["ACTUAL_OR_CLEANUP_TIME"] - clean["ACTUAL_EXIT_OR_TIME"]) <= pd.Timedelta(hours=12)
    )
    clean = clean[valid]

    if not clean.empty:
        diffs = (clean["ACTUAL_OR_CLEANUP_TIME"] - clean["ACTUAL_EXIT_OR_TIME"]).dt.total_seconds()
        marks["cleanup_offset"] = round(diffs.mean(skipna=True) / 60.0)  # minutes


    return marks


def impute_induction_prep_reversal_cleanup(row, marks):
    enter, induction, prep, knife = row[["ACTUAL_ENTER_OR_TIME", "ACTUAL_ANAESTHESIA_INDUCTION", "ACTUAL_SURGERY_PREP_TIME", "ACTUAL_KNIFE_TO_SKIN_TIME"]]
    closure, reversal, exit_, cleanup = row[["ACTUAL_SKIN_CLOSURE", "ACTUAL_PATIENT_REVERSAL_TIME", "ACTUAL_EXIT_OR_TIME", "ACTUAL_OR_CLEANUP_TIME"]]

    # --- Case A: both missing induction & prep
    if pd.notna(enter) and pd.isna(induction) and pd.isna(prep) and pd.notna(knife):
        if "induction" in marks and "prep" in marks:
            total = knife - enter
            row["ACTUAL_ANAESTHESIA_INDUCTION"] = (enter + total * marks["induction"]).round("min")
            row["ACTUAL_SURGERY_PREP_TIME"] = (enter + total * marks["prep"]).round("min")

    # --- Case B: missing prep only
    if pd.notna(induction) and pd.isna(prep) and pd.notna(knife):
        if "prep_from_induction" in marks:
            total = knife - induction
            row["ACTUAL_SURGERY_PREP_TIME"] = (induction + total * marks["prep_from_induction"]).round("min")

    # --- Case C: missing induction only
    if pd.notna(enter) and pd.isna(induction) and pd.notna(prep):
        if "induction_from_enter" in marks:
            total = prep - enter
            row["ACTUAL_ANAESTHESIA_INDUCTION"] = (enter + total * marks["induction_from_enter"]).round("min")

    # --- Case D: missing reversal
    if pd.notna(closure) and pd.isna(reversal) and pd.notna(exit_):
        if "reversal" in marks:
            total = exit_ - closure
            row["ACTUAL_PATIENT_REVERSAL_TIME"] = (closure + total * marks["reversal"]).round("min")

    # --- Case E: missing cleanup
    if pd.notna(exit_) and pd.isna(cleanup):
        if "cleanup_offset" in marks:
            row["ACTUAL_OR_CLEANUP_TIME"] = (exit_ + pd.Timedelta(minutes=marks["cleanup_offset"])).round("min")

    return row


# Step 1: get average marks
marks = compute_marks(data)

# Step 2: apply backfill
data = data.apply(lambda r: impute_induction_prep_reversal_cleanup(r, marks), axis=1)

data.iloc[:, 19:31]

,PATIENT_CALL_TIME,PATIENT_FETCH_TIME,ACTUAL_RECEPTION_IN_TIME,ACTUAL_ENTER_OR_TIME,ACTUAL_ANAESTHESIA_INDUCTION,ACTUAL_SURGERY_PREP_TIME,ACTUAL_KNIFE_TO_SKIN_TIME,ACTUAL_SKIN_CLOSURE,ACTUAL_PATIENT_REVERSAL_TIME,ACTUAL_EXIT_OR_TIME,ACTUAL_EXIT_RECOVERY_TIME,ACTUAL_OR_CLEANUP_TIME
0,2019-04-11 10:15:00,2019-04-11 10:26:00,2019-04-11 10:41:00,2019-04-11 11:04:00,2019-04-11 11:06:00,2019-04-11 11:17:00,2019-04-11 11:44:00,2019-04-11 17:31:00,2019-04-11 17:43:00,2019-04-11 17:46:00,2019-04-11 18:48:00,2019-04-11 18:04:00
1,2019-04-11 09:49:00,2019-04-11 09:55:00,2019-04-11 10:17:00,2019-04-11 11:01:00,2019-04-11 11:03:00,2019-04-11 11:08:00,2019-04-11 11:10:00,2019-04-11 11:20:00,2019-04-11 11:32:00,2019-04-11 11:32:00,2019-04-11 12:48:00,2019-04-11 11:33:00
2,2019-04-11 10:15:00,2019-04-11 10:15:00,2019-04-11 10:39:00,2019-04-11 10:58:00,2019-04-11 10:59:00,2019-04-11 11:01:00,2019-04-11 11:02:00,2019-04-11 11:21:00,2019-04-11 11:22:00,2019-04-11 11:24:00,2019-04-11 11:44:00,2019-04-11 13:35:00
3,2019-04-11 10:15:00,2019-04-11 10:18:00,2019-04-11 10:36:00,2019-04-11 10:58:00,2019-04-11 10:59:00,2019-04-11 11:09:00,2019-04-11 11:17:00,2019-04-11 13:09:00,2019-04-11 13:09:00,2019-04-11 13:27:00,2019-04-11 15:10:00,2019-04-11 13:27:00
4,2019-04-11 09:45:00,2019-04-11 10:44:00,2019-04-11 10:58:00,2019-04-11 10:58:00,2019-04-11 11:01:00,2019-04-11 11:23:00,2019-04-11 11:24:00,2019-04-11 13:17:00,2019-04-11 13:28:00,2019-04-11 13:34:00,2019-04-11 18:41:00,2019-04-11 13:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...
276858,2019-06-20 08:24:00,2019-06-20 08:26:00,2019-06-20 08:29:00,2019-06-20 09:44:00,2019-06-20 09:49:00,2019-06-20 09:50:00,2019-06-20 09:50:00,2019-06-20 10:05:00,2019-06-20 10:05:00,2019-06-20 10:07:00,2019-06-20 11:18:00,2019-06-20 10:37:00
276859,2019-06-20 08:51:00,2019-06-20 09:04:00,2019-06-20 09:13:00,2019-06-20 09:36:00,2019-06-20 09:38:00,2019-06-20 09:50:00,2019-06-20 10:19:00,2019-06-20 13:23:00,2019-06-20 13:23:00,2019-06-20 13:51:00,2019-06-20 16:14:00,2019-06-20 14:01:00
276860,2019-06-20 08:11:00,2019-06-20 08:21:00,2019-06-20 08:32:00,2019-06-20 09:20:00,2019-06-20 09:27:00,2019-06-20 09:29:00,2019-06-20 09:30:00,2019-06-20 09:50:00,2019-06-20 09:50:00,2019-06-20 09:52:00,2019-06-20 10:16:00,2019-06-20 09:53:00
276861,2019-06-20 07:44:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:34:00,2019-06-20 09:40:00,2019-06-20 09:46:00,2019-06-20 10:03:00,2019-06-20 10:04:00,2019-06-20 10:08:00,2019-06-20 10:40:00,2019-06-20 10:10:00


### Handle Rows with no Actual Data

For the purpose of our project, actual date/time data is needed to track any delays with the planned time.
By observation, we note that these rows with no actual data track to procedures marked with LOCATION == "OUT OF OT ROOMS".
These represent cases outside of operating theatres and should not be included in downstream time sequence analysis. We therefore remove them.


In [3735]:
before = len(data)
data = data[data["LOCATION"] != "OUT OF OT ROOMS"].copy()
after = len(data)

print(f"Removed {before - after} rows with LOCATION == 'OUT OF OT ROOMS' (kept {after}).")

Removed 3726 rows with LOCATION == 'OUT OF OT ROOMS' (kept 273137).


### Handle admission related columns
Some of these surgeries may be day surgeries of from the A&E, hence might not have admission data. Hence, we will replace blanks with "Not Admitted".

In [3736]:
admission_cols = ["ADMISSION_STATUS", "ADMISSION_CLASS_TYPE", 
                  "ADMISSION_TYPE", "ADMISSION_WARD", "ADMISSION_BED"]
data[admission_cols] = data[admission_cols].fillna("Not Admitted")

### Fill in missing staff data
Some surgeries are missing surgeon, anaesthetist, or diagnosis data, hence we will fill it with "Unknown" and "Not Recorded". This is because it is likely not possible for a surgery to proceed without them.

In [3737]:
clinician_cols = ["SURGEON", "ANAESTHETIST_TEAM", "ANAESTHETIST_MCR_NO"]
data[clinician_cols] = data[clinician_cols].fillna("Unknown")
data["DIAGNOSIS"] = data["DIAGNOSIS"].fillna("Not Recorded")
data

,OPERATION_ID,LOCATION,ROOM,BOOKING_DATE,CASE_STATUS,OPERATION_TYPE,EMERGENCY_PRIORITY,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,...,ADMISSION_BED,AOH,BLOOD,IMPLANT,DIAGNOSIS,CANCER_INDICATOR,TRAUMA_INDICATOR,Delay_Reason,Remarks,PATIENT_CODE_OLD
0,588456.0,Main Building OT,MBOR11,10:27:00,Final,Elective,0,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 09:50:00,...,N06A036,False,NIL,required microscope,Right Breast CA,False,False,Surgeon (e.g. Surgeon not available & etc.),0,i8fr0NKEp
1,590736.0,Main Building OT,MBOR05,14:50:00,Final,Elective,0,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:40:00,...,NASWA11,False,NIL,0,early pregnancy failure,False,False,0,0,So30dGuVZ
2,591995.0,ICL,RoomC,18:30:00,Actualised,Elective,0,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 10:55:00,...,N07B005,False,NIL,0,Not Recorded,False,False,0,0,0t2jmERej
3,590451.0,Main Building OT,MBOR04,11:49:00,Final,Elective,0,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,...,N041004,False,NIL,Need Eustachian tube ballon,EUSTACHIAN TUBE DISORDER,False,False,0,0,wDgw8UGdT
4,573666.0,Medical Center OT,MCOR03,09:00:00,Final,Elective,0,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:30:00,...,N02A025,False,NIL,0,gall bladder stone,False,False,first case havent finished,first case havent finished,WYQEJDtnk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276858,652260.0,Endoscopy Center,ENDO4,14:30:00,Actualised,Elective,0,2019-06-20 09:25:00,2019-06-20 09:25:00,2019-06-20 09:25:00,...,Not Admitted,False,NIL,0,Not Recorded,False,False,0,0,7W2rCAd5h
276859,667863.0,Main Building OT,MBOR09,13:40:00,Final,Elective,0,2019-06-20 12:45:00,2019-06-20 12:45:00,2019-06-20 12:45:00,...,N052023,False,NIL,ENT Endoscope SetENT DrillENT Neuronavigation,Pituitary Tumour,False,False,Resequencing of cases,0,5CUH725Gw
276860,667473.0,Endoscopy Center,ENDO2,16:16:00,Actualised,Elective,0,2019-06-20 08:30:00,2019-06-20 08:30:00,2019-06-20 08:30:00,...,N042041,False,NIL,0,Follow up on varices.,False,False,0,0,g9q0nQP79
276861,665881.0,Medical Center OT,MCOR05,17:52:00,Final,Elective,0,2019-06-20 09:20:00,2019-06-20 09:20:00,2019-06-20 09:20:00,...,NASWC82,False,NIL,0,left eye cataract,False,False,0,0,6RhdSQCfM


### Drop remaining missing rows
After filling in the missing values that we are able to fill, there are some columns that are still missing data. We will thus drop them as they make up a very small portion of our overall data.

In [3738]:
before = len(data)
data.dropna(inplace=True)
after = len(data)

print(f"Dropped {before - after} rows (kept {after}).")
data

Dropped 16442 rows (kept 256695).


,OPERATION_ID,LOCATION,ROOM,BOOKING_DATE,CASE_STATUS,OPERATION_TYPE,EMERGENCY_PRIORITY,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,...,ADMISSION_BED,AOH,BLOOD,IMPLANT,DIAGNOSIS,CANCER_INDICATOR,TRAUMA_INDICATOR,Delay_Reason,Remarks,PATIENT_CODE_OLD
0,588456.0,Main Building OT,MBOR11,10:27:00,Final,Elective,0,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 09:50:00,...,N06A036,False,NIL,required microscope,Right Breast CA,False,False,Surgeon (e.g. Surgeon not available & etc.),0,i8fr0NKEp
1,590736.0,Main Building OT,MBOR05,14:50:00,Final,Elective,0,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:40:00,...,NASWA11,False,NIL,0,early pregnancy failure,False,False,0,0,So30dGuVZ
2,591995.0,ICL,RoomC,18:30:00,Actualised,Elective,0,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 10:55:00,...,N07B005,False,NIL,0,Not Recorded,False,False,0,0,0t2jmERej
3,590451.0,Main Building OT,MBOR04,11:49:00,Final,Elective,0,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,...,N041004,False,NIL,Need Eustachian tube ballon,EUSTACHIAN TUBE DISORDER,False,False,0,0,wDgw8UGdT
4,573666.0,Medical Center OT,MCOR03,09:00:00,Final,Elective,0,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:30:00,...,N02A025,False,NIL,0,gall bladder stone,False,False,first case havent finished,first case havent finished,WYQEJDtnk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276856,656953.0,Main Building OT,MBOR01,14:07:00,Final,Elective,0,2019-06-20 09:50:00,2019-06-20 09:50:00,2019-06-20 09:50:00,...,N02A031,False,NIL,0,Right renal stones,False,False,"Pre-Op (e.g. no consent, investigation, blood ...",0,u5YGiL9aG
276858,652260.0,Endoscopy Center,ENDO4,14:30:00,Actualised,Elective,0,2019-06-20 09:25:00,2019-06-20 09:25:00,2019-06-20 09:25:00,...,Not Admitted,False,NIL,0,Not Recorded,False,False,0,0,7W2rCAd5h
276859,667863.0,Main Building OT,MBOR09,13:40:00,Final,Elective,0,2019-06-20 12:45:00,2019-06-20 12:45:00,2019-06-20 12:45:00,...,N052023,False,NIL,ENT Endoscope SetENT DrillENT Neuronavigation,Pituitary Tumour,False,False,Resequencing of cases,0,5CUH725Gw
276860,667473.0,Endoscopy Center,ENDO2,16:16:00,Actualised,Elective,0,2019-06-20 08:30:00,2019-06-20 08:30:00,2019-06-20 08:30:00,...,N042041,False,NIL,0,Follow up on varices.,False,False,0,0,g9q0nQP79


### Sanity Check: Chronological Order of Planned and Actual Times

After imputing missing values, we perform a **sanity check** on the `planned_cols`, and `actual_cols`.  
For each row, the planned and actual timestamps should follow the natural chronological order of surgical workflow:

Rows where these values are **not monotonically increasing (≥)** are considered invalid,  
since they break the logical sequence of events.  
Such rows are dropped from the dataset.

This ensures the dataset maintains internal consistency, and weeds out potential data entry errors.

In [ ]:
def is_monotonic(row, cols):
    """Return True if all non-null values in cols are monotonically increasing (>=)."""
    vals = [row[c] for c in cols if c in row.index and pd.notna(row[c])]
    return all(x <= y for x, y in zip(vals, vals[1:]))

# Apply filter for planned
before = len(data)
data = data[data.apply(lambda r: is_monotonic(r, planned_cols), axis=1)]
after = len(data)

print(f"Dropped {before - after} rows where planned times were not monotonically increasing.")

# Apply filter for actual
before = len(data)
data = data[data.apply(lambda r: is_monotonic(r, actual_cols), axis=1)]
after = len(data)

print(f"Dropped {before - after} rows where actual times were not monotonically increasing.")

### View current state of dataframe
Currently, the dataset no longer contains any missing data, and thus we are able to proceed with the next steps.

In [3739]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256695 entries, 0 to 276861
Data columns (total 54 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   OPERATION_ID                   256695 non-null  float64       
 1   LOCATION                       256695 non-null  object        
 2   ROOM                           256695 non-null  object        
 3   BOOKING_DATE                   256695 non-null  object        
 4   CASE_STATUS                    256695 non-null  object        
 5   OPERATION_TYPE                 256695 non-null  object        
 6   EMERGENCY_PRIORITY             256695 non-null  object        
 7   PLANNED_PATIENT_CALL_TIME      256695 non-null  object        
 8   PLANNED_PATIENT_FETCH_TIME     256695 non-null  object        
 9   PLANNED_RECEPTION_IN_TIME      256695 non-null  object        
 10  PLANNED_ENTER_OR_TIME          256695 non-null  object        
 11  PLANN

## Handle Duplicate Data
Important to remove to prevent bias in our AI/ML solution

### Check for duplicate rows
This is to see if our dataset contains any rows that are completely identical. This means that the same surgery has been accidentally logged twice. We want to avoid having this in our dataset as it would cause our analysis in the future to skew.

In [3740]:
data.duplicated().sum()

3

### Drop duplicate rows
We identified 3 duplicate rows, and hence we will want to drop them. 

In [3741]:
data = data.drop_duplicates()

In [3742]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256692 entries, 0 to 276861
Data columns (total 54 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   OPERATION_ID                   256692 non-null  float64       
 1   LOCATION                       256692 non-null  object        
 2   ROOM                           256692 non-null  object        
 3   BOOKING_DATE                   256692 non-null  object        
 4   CASE_STATUS                    256692 non-null  object        
 5   OPERATION_TYPE                 256692 non-null  object        
 6   EMERGENCY_PRIORITY             256692 non-null  object        
 7   PLANNED_PATIENT_CALL_TIME      256692 non-null  object        
 8   PLANNED_PATIENT_FETCH_TIME     256692 non-null  object        
 9   PLANNED_RECEPTION_IN_TIME      256692 non-null  object        
 10  PLANNED_ENTER_OR_TIME          256692 non-null  object        
 11  PLANN

## Deep cleaning each column
Looking into each individual column to clean up most of the free text portions. Please add more cleaning as we go, as there is quite alot to sieve through and I dont think i caught it all.

In [3743]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256692 entries, 0 to 276861
Data columns (total 54 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   OPERATION_ID                   256692 non-null  float64       
 1   LOCATION                       256692 non-null  object        
 2   ROOM                           256692 non-null  object        
 3   BOOKING_DATE                   256692 non-null  object        
 4   CASE_STATUS                    256692 non-null  object        
 5   OPERATION_TYPE                 256692 non-null  object        
 6   EMERGENCY_PRIORITY             256692 non-null  object        
 7   PLANNED_PATIENT_CALL_TIME      256692 non-null  object        
 8   PLANNED_PATIENT_FETCH_TIME     256692 non-null  object        
 9   PLANNED_RECEPTION_IN_TIME      256692 non-null  object        
 10  PLANNED_ENTER_OR_TIME          256692 non-null  object        
 11  PLANN

### Inspect Location
No problems.

#### Unique values analysis

In [3744]:
inspect_column(data, "LOCATION", top_n=30)

Column: LOCATION
Unique values: 9

Top value counts:
LOCATION
Main Building OT          79518
Endoscopy Center          63101
Medical Center OT         55508
Kent Ridge Wing OT        22509
ICL                       20337
DDI                       15523
Urology                     136
AH Endoscopy Center          54
AH Day Surgery Theatre        6
Name: count, dtype: int64


### Inspect Room
No problems.

#### Unique values analysis

In [3745]:
inspect_column(data, "ROOM", top_n=30)

Column: ROOM
Unique values: 56

Top value counts:
ROOM
ENDO4       13359
ENDO2       11350
MBOR08      11211
MBOR09      10527
ENDO3        9687
RoomC        9606
MCOR05       9429
ENDO6        9393
ENDO1        9283
MCOR04       7639
RoomB        7006
MCOR06       6697
KRWOR1       6618
ENDO5        6481
DDIXA001     6315
MBOR18       6210
MBOR14       6129
MCOR10       5097
DDIXA004     5081
MCOR03       4948
MCOR09       4685
MBOR07       4553
MCOR07       4433
MCOR02       4416
MBOR05       4401
MBOR06       4264
DDIXA007     4127
MCOR08       4103
MCOR01       4061
RoomA        3725
Name: count, dtype: int64


### Inspect case status
No problems.

#### Unique values analysis

In [3746]:
inspect_column(data, "CASE_STATUS", top_n=30)

Column: CASE_STATUS
Unique values: 5

Top value counts:
CASE_STATUS
Final         139604
Actualised    116805
Booked           159
Missed           123
Rebooking          1
Name: count, dtype: int64


### Inspect OPERATION_TYPE
No problems.

#### UVA

In [3747]:
inspect_column(data, "OPERATION_TYPE", top_n=30)

Column: OPERATION_TYPE
Unique values: 2

Top value counts:
OPERATION_TYPE
Elective     215783
Emergency     40909
Name: count, dtype: int64


### Inspect Emergency Priority
No problems.

#### UVA

In [3748]:
inspect_column(data, "EMERGENCY_PRIORITY", top_n=30)

Column: EMERGENCY_PRIORITY
Unique values: 8

Top value counts:
EMERGENCY_PRIORITY
0      215783
P2      24787
P1       9639
P2A      2751
P0       2308
P3        970
P3a       380
P3b        74
Name: count, dtype: int64


### Inspect Patient Code
No problems.

#### UVA

In [3749]:
inspect_column(data, "PATIENT_CODE", top_n=30)

Column: PATIENT_CODE
Unique values: 155322

Top value counts:
PATIENT_CODE
mrSK5m9nqRlG[@dG    64
d<P8DK2?5odu8As6    63
EJ6QB8qKDtZR3&LP    39
a9a5R8heBV8Gy02e    38
S57v9vPZ64irK5ko    36
T4+jrog6Rnj3kWRv    34
>oPUp7[6P7kZx33D    32
6#66j3XPCph&DeQU    32
io5[ei}@8b7d5t$i    32
435T3j@UE4Ghkzpf    31
}wX^sJ4ou0+Wyi{&    31
AwXDGce4$L3N8dX6    31
58}cNxQeH8MlvNn6    31
ptEf3fen^L?3Rxm5    31
N5l}@eEs8nM005V2    29
pth3}9j9ZkEia9$y    29
XLyDnhn5teXzEZ8W    29
[?d#5CHonA4MY@h5    28
Nv#Um78atsyUuWDg    28
?53&jgEQiWbq]uG6    28
8y&6&Wu07xicst2D    28
L8id0<f+flK5Q2Fq    28
sPHpnBW77GKDQA9f    27
8LFmDZhUtKDoP2sT    27
cT7dr7v}KvtoAXu8    27
qLaR0<GgTuhQRsw6    27
9xnrcUn0Bmui4edX    27
SxEwHv4w2pZuNxUf    27
dxlvF7Gr2lYFd6#v    27
wtlQ6lAU{}Wop0n;    27
Name: count, dtype: int64


### Inspect Nature
Removed this column entirely, and created a legend(can be found below) to map SURGICAL_CODE to NATURE, as they are the same thing.

#### UVA

In [3750]:
inspect_column(data, "NATURE", top_n=30)

Column: NATURE
Unique values: 9609

Top value counts:
NATURE
Intestine/Stomach, Upper GI endoscopy with / without biopsy (SF701I)                                                                                      20739
Colon, Colonoscopy (diagnostic), fibreoptic with/without biopsy (SF702C)                                                                                  12715
Colon, Colonoscopy (diagnostic), fibreoptic with/without biopsy (SF702C); Intestine/Stomach, Upper GI endoscopy with / without biopsy (SF701I)            11551
Dilation Of Cervix (SIC004)                                                                                                                                6556
Skin and Subcutaneous Tissue, Deep>3cm/Extensive Contaminated Wound, Debridement (SA811S)                                                                  6548
Coronary Angio. [ICL] (ISD811H)                                                                                                            

In [3751]:
data, nature_legend = build_operation_legend_and_drop_nature(
    data,
    code_col="SURGICAL_CODE",
    nature_col="NATURE",
    drop_nature=True,           
    keep_title_case_copy=True    
)

In [3752]:
print(data.columns)              
nature_legend.drop(columns='operation_name_title', inplace=True)
nature_legend

Index(['OPERATION_ID', 'LOCATION', 'ROOM', 'BOOKING_DATE', 'CASE_STATUS',
       'OPERATION_TYPE', 'EMERGENCY_PRIORITY', 'PLANNED_PATIENT_CALL_TIME',
       'PLANNED_PATIENT_FETCH_TIME', 'PLANNED_RECEPTION_IN_TIME',
       'PLANNED_ENTER_OR_TIME', 'PLANNED_ANAESTHESIA_INDUCTION',
       'PLANNED_SURGERY_PREP_TIME', 'PLANNED_KNIFE_TO_SKIN_TIME',
       'PLANNED_SKIN_CLOSURE', 'PLANNED_PATIENT_REVERSAL_TIME',
       'PLANNED_EXIT_OR_TIME', 'PLANNED_EXIT_RECOVERY_TIME',
       'PLANNED_OR_CLEANUP_TIME', 'PATIENT_CALL_TIME', 'PATIENT_FETCH_TIME',
       'ACTUAL_RECEPTION_IN_TIME', 'ACTUAL_ENTER_OR_TIME',
       'ACTUAL_ANAESTHESIA_INDUCTION', 'ACTUAL_SURGERY_PREP_TIME',
       'ACTUAL_KNIFE_TO_SKIN_TIME', 'ACTUAL_SKIN_CLOSURE',
       'ACTUAL_PATIENT_REVERSAL_TIME', 'ACTUAL_EXIT_OR_TIME',
       'ACTUAL_EXIT_RECOVERY_TIME', 'ACTUAL_OR_CLEANUP_TIME', 'PATIENT_CODE',
       'SURGICAL_CODE', 'DISCIPLINE', 'SURGEON', 'ANAESTHETIST_TEAM',
       'ANAESTHETIST_MCR_NO', 'ANESTHESIA', 'EQUIPMENT',

,operation_code,operation_name
0,0050741; ISD705H,biopsy (under fluoroscopic guidance) (0050741)...
1,0050741; ISD706H,biopsy (under fluoroscopic guidance) (0050741)...
2,0050741; ISD734H,biopsy (under fluoroscopic guidance) (0050741)...
3,0050741; ISD734H; ZSD706H,biopsy (under fluoroscopic guidance) (0050741)...
4,0050741; ISD815H,biopsy (under fluoroscopic guidance) (0050741)...
...,...,...
9192,ZSD802H; ISD706H,eps with ablation [icl] (zsd802h); icd implant...
9193,ZSD802H; ISD716H,eps with ablation [icl] (zsd802h); perm. cardi...
9194,ZSD802H; ZSD706H,eps with ablation [icl] (zsd802h); biventricul...
9195,ZSD810H,cor. angio with angioplasty - simple [icl]


In [3753]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256692 entries, 0 to 276861
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   OPERATION_ID                   256692 non-null  float64       
 1   LOCATION                       256692 non-null  object        
 2   ROOM                           256692 non-null  object        
 3   BOOKING_DATE                   256692 non-null  object        
 4   CASE_STATUS                    256692 non-null  object        
 5   OPERATION_TYPE                 256692 non-null  object        
 6   EMERGENCY_PRIORITY             256692 non-null  object        
 7   PLANNED_PATIENT_CALL_TIME      256692 non-null  object        
 8   PLANNED_PATIENT_FETCH_TIME     256692 non-null  object        
 9   PLANNED_RECEPTION_IN_TIME      256692 non-null  object        
 10  PLANNED_ENTER_OR_TIME          256692 non-null  object        
 11  PLANN

### Inspect Surgical Code
Extension of NATURE.

#### UVA

In [3754]:
inspect_column(data, "SURGICAL_CODE", top_n=30)

Column: SURGICAL_CODE
Unique values: 9197

Top value counts:
SURGICAL_CODE
SF701I            20739
SF702C            16908
SF702C; SF701I    15610
SP834U             6981
SIC004             6556
SA811S             6548
ISD811H            5615
SL809L             4503
SL808L             4402
ZSD810H            3452
XSD722V            3215
LL808L             3164
LL809L             3040
ISD714H            2525
SD707A             1893
SA701S             1803
SH808P             1767
SF710C             1748
SB810K             1574
ISD810H            1486
SA841S             1307
SF718B             1279
SA702S             1196
XSF706A            1195
SD812H             1158
SA803S             1136
XSD721V            1096
ZSD714H            1057
LF849A             1056
SL801V             1038
Name: count, dtype: int64


### Inspect discipline
No problems.

#### UVA

In [3755]:
inspect_column(data, "DISCIPLINE", top_n=30)

Column: DISCIPLINE
Unique values: 20

Top value counts:
DISCIPLINE
Surgery                          57230
Gastroenterology & Hepatology    35872
Obstetrics & Gynaecology         24817
Orthopaedic Surgery              24131
Ophthalmology                    21975
Cardiac                          19583
Radiology                        15528
CardiacThoracic&Vascular Surg    14785
Hand&Reconstructive Microsurg    12262
Otolaryngology&Head&Neck Surg     9046
Urology                           7219
Paediatric Surgery                5099
Medicine                          3273
Dental                            3002
Psychological Medicine            1266
Paediatrics                        818
Anaesthesia                        611
Transplant                         152
Haematology - Oncology              18
Neonatology                          5
Name: count, dtype: int64


### Inspect Surgeon
No problems.

#### UVA

In [3756]:
inspect_column(data, "SURGEON", top_n=30)

Column: SURGEON
Unique values: 1233

Top value counts:
SURGEON
Unknown             15520
lHwi02ZeE3K2VunU     4224
ZEFBvq6RyyUi06&+     4004
mbQ;KNz{$02aCxsz     3673
haaeD5Q>4l&5cyMs     3071
nlJsN6bxb860fuvn     3066
96T?5Mj8VSelR}Cl     2906
aWZlndEB5y?M]03P     2750
C29nEfgqQBB7xVT?     2600
nei0JobdGxcE65t6     2470
RMDcp3MhHDP7tCgg     2359
T8Qv5G#&i6Qb$&q0     2317
qzEBDv<3hxja@xlk     2287
<r7whvGE0gU6QKKt     2269
#w0VHw03M#Wz2qtv     2122
}z#JLao8aJTQxqL<     2040
exKU23ieN7zJsGLZ     2016
50c2z5Lc&m7Z2TaK     1950
ci2iezjzZ}L[{QN6     1885
M[;D[QHC?Skd}JC8     1782
k@NSCW#4VzNuD5p8     1748
Xdph7$0RR^LtG27y     1642
R8Xq6zPliFdBfK$8     1641
06eDEgGBCh{gowXU     1601
W5{6rF02PmZX+ybv     1518
CmMlPoq8055XLXsw     1493
LrZPovtT5MkTGz9[     1469
]FlWay&b[5b[v<oj     1375
y[;kyh[m]CLsJM2A     1365
7rV86GcmAbuwptu0     1354
Name: count, dtype: int64


### Inspect ANAESTHETIST_TEAM
No problems.

#### UVA

In [3757]:
inspect_column(data, "ANAESTHETIST_TEAM", top_n=30)

Column: ANAESTHETIST_TEAM
Unique values: 21399

Top value counts:
ANAESTHETIST_TEAM
Unknown                                                             127110
d053551ad4c28a2fb932ba43e6b1d113c638a50ef979df413d055fbd5f0f3192      2297
90697bd90f1e98133b3baf52d7d8bad94fae0f82182ccbce369b231d0e9aad81      1496
c3d2524de7f2c23598e5af6961683a7e44e61bb39fa6ccc5cfe5f236baf44649      1254
b8dd61fc03d6e3a642dfb099851f01cdc80de504245f49c58129dae4b4865a5d      1015
69e7e4a0e4d5bc7eb3852b47b0a1828bdab6115e8bd860cd44c4ebc354a17707       890
5f1d6a174c8cb3052fed44ccc95e00ae34c73c4e2eda8465422877dd6b08ae2a       747
670c0eb324e240959bf85b5d91f5abee71f7af88e1cc3cb3de887ba63bf32f91       741
d209a9569b8f43d22756e5ea8b8a75df973b9a1c47e60fe798614a959c5fe576       601
3e2583cfd30a4c8732acd737262803aaf1b765d638fb94c1fecf3302680801df       596
4baa32452676f4c55f7a42b895c59c41af1dc17a33ea3bba597526007b766310       594
b321002af2aa9a977e25c26492b6d2de51847c3dbbaf7792845e09b75bb222fa       572
e51218bf6137b06a

### INSPECT ANAESTHETIST_MCR_NO
No problems.

#### UVA

In [3758]:
inspect_column(data, "ANAESTHETIST_MCR_NO", top_n=30)

Column: ANAESTHETIST_MCR_NO
Unique values: 398

Top value counts:
ANAESTHETIST_MCR_NO
Unknown                                                             127420
aabd0c05e0c4e9b6bfa368698260838d288a941f1b16bd4ada4b4ef8abdd9bbc      2428
b00c197ef120c6d366079d1d280ad75fa4956a4856609b4c7b22964e76b2fe80      2065
502af80887f9dad24b15c17c60e6e4a1c4aff7055acab561e970eb9b88a26520      2040
336fdaba1f1c9345f491a00dd3271735dad3e04e2346d423e865f41bcb1c4e50      2000
fa8be41538144d4b7c35e1daee25a14619bed1c76feef34df97625dd971690af      1956
392d9547cf7c381e067ba6b48329ee24d8989d89a19355f3a0da1c727532632e      1895
7bb2e45c6fd6182f126613c5312ae2cb698fce0b7be26639d2be9629b09b1c92      1871
5a40b6fd919d12b44462644d68c104c541fbecd0086225864bcbfedbfc285a75      1826
2111d59051e5844a4ef523b9aa4db854d2783662d557ffb169152f950a6bf3a9      1727
4eeecfeb436a3c9cf98b312a0fc1e9b3d44ce3b146b86634b4cc5526e2f1a53a      1712
4dc6309a08fde9e4d31789934b6bde065615a580995448a9b400800b7e6bee28      1703
a172aa594dcf84

### INSPECT ANESTHESIA
No problems.

#### UVA

In [3759]:
inspect_column(data, "ANESTHESIA", top_n=30)

Column: ANESTHESIA
Unique values: 7

Top value counts:
ANESTHESIA
GA                            108445
Sedation                       63448
LA                             53608
NIL                            20106
RA                              9083
Monitored Anaesthesia Care      1945
LA (Standby)                      57
Name: count, dtype: int64


### Inspect EQUIPMENT
Removed #NUH_ or #NUH from all entries, as well as alphabetically ordered the equipment such that even if they were in different orders, they would appear under the same unique value.

#### UVA

In [3760]:
inspect_column(data, "EQUIPMENT", top_n=30)

Column: EQUIPMENT
Unique values: 550

Top value counts:
EQUIPMENT
0                                                              235766
#NUH_Image Intensifier                                           8130
Image Intensifier                                                5200
#NUH_Mini II                                                     1525
Mini II                                                          1186
#NUH_Da Vinci Robotics                                            452
Da Vinci Robotics                                                 289
#NUH_Image Intensifier; #NUH_Jackson Table                        219
#NUH_Hand Microscope                                              184
Image Intensifier; Jackson Table                                  177
#NUH_Microscope                                                   159
Hand Microscope                                                   149
Microscope                                                        143
#NUH_Image Intensifier; 

In [3761]:
synonyms = {}
data = clean_equipment(
    data,
    col="EQUIPMENT",
    tags_to_strip=(r"#nuh",),        
    unknown_vals=("0","na","n/a","-","null","nan",""),
    synonym_map=synonyms
)

# Inspect results
inspect_column(data, "EQUIPMENT", top_n=30)

Column: EQUIPMENT
Unique values: 229

Top value counts:
EQUIPMENT
0                                                               235766
image intensifier                                                13331
mini ii                                                           2711
da vinci robotics                                                  741
image intensifier; jackson table                                   500
hand microscope                                                    333
amsco table; image intensifier                                     317
microscope                                                         302
amsco table; image intensifier; microscope                         202
image intensifier; microscope                                      168
image intensifier; traction table                                  165
image intensifier; jackson table; microscope                       142
head light; image intensifier; jackson table                       137
hand table 

### Inspect ADMISSION_STATUS
No problems.

#### UVA

In [3762]:
inspect_column(data, "ADMISSION_STATUS", top_n=30)

Column: ADMISSION_STATUS
Unique values: 7

Top value counts:
ADMISSION_STATUS
Discharged      211180
Actual           37437
Not Admitted      7877
Planned            137
Cancelled           59
1518656227           1
1518637975           1
Name: count, dtype: int64


### Inspect ADMISSION_CLASS_TYPE
No problems.

#### UVA

In [3763]:
inspect_column(data, "ADMISSION_CLASS_TYPE", top_n=30)

Column: ADMISSION_CLASS_TYPE
Unique values: 21

Top value counts:
ADMISSION_CLASS_TYPE
Subsidised         76722
Class C            51328
Class B2           39693
Private            18105
Class A            12210
Non Resident       10961
Not Admitted        7876
Subsidised PR       5047
Class B1            4829
Private for RF      4707
Class C for RF      4340
Class A for RF      3669
Non Resident B1     3567
Private for PR      3524
Class C for PR      3329
Class A for PR      2603
Class B2 for PR     2603
Class B1 for PR      699
Class B1 for RF      487
Class B2 for RF      391
Discharged             2
Name: count, dtype: int64


### Inspect ADMISSION_TYPE
No problems.

#### UVA

In [3764]:
inspect_column(data, "ADMISSION_TYPE", top_n=30)

Column: ADMISSION_TYPE
Unique values: 13

Top value counts:
ADMISSION_TYPE
Emergency         71129
Day Surgery       64878
Endoscopy         47257
Same Day Adm.     45516
Elective inpa     13838
Not Admitted       7880
DS turn Inpat      5225
Day Surgery OP      673
Newborn             216
Transfer             62
Technical Adm        16
Class A for PR        1
Class C               1
Name: count, dtype: int64


### Inspect ADMISSION_WARD
No problems.

#### UVA

In [3765]:
inspect_column(data, "ADMISSION_WARD", top_n=30)

Column: ADMISSION_WARD
Unique values: 100

Top value counts:
ADMISSION_WARD
NWASW           54122
NCKED           45556
NW2A            30914
NW63            10278
Not Admitted     9306
NW43             7985
NW51             7651
NW41             6768
NW28             5853
NW52             5361
NW64             5209
NW48             4572
NW44             4268
NW47             3861
NWICL            3516
NW9B             3395
NWEDS            3370
NW6A             3230
NW54             3212
NW7A             3143
NW9A             2659
NW6B             2639
NW55             2482
NW7B             2403
NW42             2335
NW27             2063
NW56             1768
NCZCOL           1525
NW33             1438
NW22             1368
Name: count, dtype: int64


### Inspect ADMISSION_BED
No problems.

#### UVA

In [3766]:
inspect_column(data, "ADMISSION_BED", top_n=30)

Column: ADMISSION_BED
Unique values: 1614

Top value counts:
ADMISSION_BED
Not Admitted    57118
NASWC82          9299
NASWC31          7093
NEDS001          3366
NASWB01          1634
NASWB25          1430
NASWB21          1379
NASWB19          1358
NASWB22          1356
NASWB20          1344
NASWB26          1324
NASWB30          1298
NASWB15          1271
N063DL01         1265
NASWB10          1263
NASWB09          1254
NASWB29          1249
NASWB16          1247
NASWB23          1226
NASWB08          1219
NASWB28          1209
NASWB14          1207
NASWB27          1184
NASWB11          1160
NASWB17          1157
NASWB07          1102
NASWB24          1093
NASWB18          1084
NASWB13          1069
NASWB12           993
Name: count, dtype: int64


### Inspect AOH
Fixed True False.

#### UVA

In [3767]:
inspect_column(data, "AOH", top_n=30)

Column: AOH
Unique values: 4

Top value counts:
AOH
False      256599
True           91
N05A006         1
N052031         1
Name: count, dtype: int64


In [3768]:
data = normalize_text(data, "AOH", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "AOH", top_n=30)

Column: AOH
Unique values: 4

Top value counts:
AOH
false      256599
true           91
n05a006         1
n052031         1
Name: count, dtype: int64


### Inspect BLOOD
No problems.

#### UVA

In [3769]:
inspect_column(data, "BLOOD", top_n=30)

Column: BLOOD
Unique values: 8

Top value counts:
BLOOD
NIL               238492
Blood - 2 pint     13580
GXM                 4593
TXS                   14
Blood - 1 pint         5
Blood - 4 pint         5
False                  2
Blood - 3 pint         1
Name: count, dtype: int64


### Inspect IMPLANT
remove 'yes' remove 'x1' remove multiple white spaces, leading and trailing whitespaces and symbols.

#### UVA

In [3770]:
inspect_column(data, "IMPLANT", top_n=30)

Column: IMPLANT
Unique values: 16909

Top value counts:
IMPLANT
0                                     232573
na                                       566
-                                        518
NIL                                      431
Stent                                    398
YES X1 PRESSURE WIRE AND X1 STENT.       310
mesh                                     227
YES                                      199
Mesh                                     195
STENT                                    148
NIL.                                     137
nil                                      133
MESH                                     123
YES X1 IMPLANT.                          122
Nil                                       91
YES X1 STENT.                             71
PRESSURE WIRE                             59
Smith and Nephew                          52
Camera system 1488                        51
YES X1 IMPLANT                            49
Smith & Nephew                      

In [3771]:
data = normalize_text(data, "IMPLANT", unknown_vals=["0", "na", "n/a", "-", "null", "nan", "", "nil", "nil."])
data["IMPLANT"] = (
    data["IMPLANT"]
    .astype(str)
    .str.strip(" ;,.-")
    .str.replace(r"\bx\d+\b", "", regex=True)
    .str.replace(r"\byes\b", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)    
    .str.strip()
    .str.replace("&", "and", regex=False)
)
inspect_column(data, "IMPLANT", top_n=30)

Column: IMPLANT
Unique values: 15941

Top value counts:
IMPLANT
0                          233942
                              772
stent                         695
mesh                          547
pressure wire and stent       338
pressure wire                 218
implant                       193
smith and nephew              166
camera system 1488            102
valve                         100
device                         79
graft                          78
operating microscope           76
vac dressing                   73
k wires                        68
flat jackson table             66
zimmer nexgen                  57
vac                            56
, valve                        54
phenol                         54
stryker scorpio nrg            51
stryker                        44
navigation                     43
, device                       42
k wire                         41
medtronic solera               34
smith nephew                   32
arthrex           

### Inspect DIAGNOSIS
just normal standardisation

#### UVA

In [3772]:
inspect_column(data, "DIAGNOSIS", top_n=30)

Column: DIAGNOSIS
Unique values: 60544

Top value counts:
DIAGNOSIS
Not Recorded                      89450
cataract                           4717
right cataract                     3635
left cataract                      3593
subfert                            3029
Subfert                            2217
End stage renal failure            1904
CORONARY ARTERY DISEASE            1476
Peripheral vascular disease        1249
Cataract                           1232
END STAGE RENAL FAILURE             934
subfertility                        877
ISCHEMIC HEART DISEASE              825
Right cataract                      781
Left cataract                       776
dental caries                       672
Left breast cancer                  636
Right breast cancer                 612
Gall bladder stones                 562
Acute appendicitis                  513
Previous LSCS                       497
Phimosis                            463
acute appendicitis                  450
Impacted wis

In [3773]:
data = normalize_text(data, "DIAGNOSIS", unknown_vals=["0", "na", "n/a", "-", "null", "nan", "", "nil"])
inspect_column(data, "DIAGNOSIS", top_n=100)

Column: DIAGNOSIS
Unique values: 51315

Top value counts:
DIAGNOSIS
not recorded                    89450
cataract                         5968
subfert                          5442
right cataract                   4525
left cataract                    4470
                                ...  
unplanned pregnancy               162
supraventricular tachycardia      161
cyst                              158
gallstone                         158
right distal radius fracture      158
Name: count, Length: 100, dtype: int64


### Inspect CANCER_INDICATOR
just normal standardisation

#### UVA

In [3774]:
inspect_column(data, "CANCER_INDICATOR", top_n=30)

Column: CANCER_INDICATOR
Unique values: 14

Top value counts:
CANCER_INDICATOR
False                                                                                   251038
True                                                                                      5637
CI Pinless x1 ?                                                                              4
CI Pinless x1 in house?                                                                      3
Surgical fixation of left femur and left tibia and left lower limb wound debridement         1
HP x7  1 sets ?                                                                              1
CI Pinless x (x1 in house) ?                                                                 1
Gen Spine x1-                                                                                1
CI Pinless ?                                                                                 1
CI Pinless (x1 in house) ?                                        

In [3775]:
data = normalize_text(data, "CANCER_INDICATOR", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "CANCER_INDICATOR", top_n=30)

Column: CANCER_INDICATOR
Unique values: 14

Top value counts:
CANCER_INDICATOR
false                                                                                   251038
true                                                                                      5637
ci pinless x1 ?                                                                              4
ci pinless x1 in house?                                                                      3
surgical fixation of left femur and left tibia and left lower limb wound debridement         1
hp x7 1 sets ?                                                                               1
ci pinless x (x1 in house) ?                                                                 1
gen spine x1                                                                                 1
ci pinless ?                                                                                 1
ci pinless (x1 in house) ?                                        

### Inspect TUMOR_INDICATOR
just normal standardisation

#### UVA

In [3776]:
inspect_column(data, "TRAUMA_INDICATOR", top_n=30)

Column: TRAUMA_INDICATOR
Unique values: 10

Top value counts:
TRAUMA_INDICATOR
False                                                             256573
True                                                                 104
PFC x5 in house?                                                       7
HP x7 ?                                                                2
PFC x5  (in house) ?                                                   1
HP x7  1 sets ?                                                        1
PFC x5 in house (in house) ?                                           1
Mono screws x1-                                                        1
PA/PMI Set?                                                            1
Oesophagogastroduodenoscopy and Common bile duct stent removal         1
Name: count, dtype: int64


In [3777]:
data = normalize_text(data, "TRAUMA_INDICATOR", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "TRAUMA_INDICATOR", top_n=30)


Column: TRAUMA_INDICATOR
Unique values: 10

Top value counts:
TRAUMA_INDICATOR
false                                                             256573
true                                                                 104
pfc x5 in house?                                                       7
hp x7 ?                                                                2
pfc x5 (in house) ?                                                    1
hp x7 1 sets ?                                                         1
pfc x5 in house (in house) ?                                           1
mono screws x1                                                         1
pa pmi set?                                                            1
oesophagogastroduodenoscopy and common bile duct stent removal         1
Name: count, dtype: int64


### Inspect Delay_Reason
No problems.

#### UVA

In [3778]:
inspect_column(data, "Delay_Reason", top_n=30)

Column: Delay_Reason
Unique values: 5378

Top value counts:
Delay_Reason
0                                                                              204532
Admin (e.g. registration and etc.)                                               9870
Surgeon (e.g. Surgeon not available & etc.)                                      5380
Resequencing of cases                                                            4579
e case                                                                           3738
MCOT End of AM Elective List: 830am-1pm (Last scheduled AM case of the day)      3437
OR not ready (Cleaning, setting up OT table / equipment)                         2077
Patient (e.g. new medical condition, URTI, cancellation)                         1449
Pre-Op (e.g. no consent, investigation, blood , Xray)                            1395
emergency case                                                                   1345
MBOT End of Elective List: 830am-530pm (Last scheduled case of the 

### Inspect Remarks
No problems.

#### UVA

In [3779]:
inspect_column(data, "Remarks", top_n=30)

Column: Remarks
Unique values: 9040

Top value counts:
Remarks
0                    231700
e case                 4286
emergency case         1505
no delay                934
emergency               717
ecase                   506
e-case                  406
E case                  391
EOT                     329
eot                     315
different surgeon       297
.                       292
e case                  282
no delays               217
E CASE                  178
na                      178
eot case                152
aoh                     137
AOH                      96
em case                  95
Emergency case           91
p1                       88
additional case          73
e list                   69
others                   69
pm list                  66
emergency                64
p1 case                  63
EOT case                 62
emergency op             60
Name: count, dtype: int64


Save file from cleaning steps above into a seperate file (change file name if required)

In [3780]:
output_path = "oots-data-cleaning-1.xlsx"
data.to_excel(output_path, index=False)

print(f"Saved to {output_path}")

Saved to oots-data-cleaning-1.xlsx


In [3781]:
import re
import string
import pandas as pd
from collections import Counter

Find the highest frequency of words, bigrams, and trigrams to be used in taxonomy for categorisation

In [3782]:
INPUT_XLSX = "oots-data-cleaning-1.xlsx"

DELAY_COL = "Delay_Reason"

OUT_WORDS    = "oots-delay-wordcount-cleaned-1.csv"
OUT_BIGRAMS  = "oots-delay-bigram-count-cleaned-1.csv"
OUT_TRIGRAMS = "oots-delay-trigram-count-cleaned-1.csv"

In [3783]:
df = pd.read_excel(INPUT_XLSX)

if DELAY_COL not in df.columns:
    raise KeyError(f"'{DELAY_COL}' not found. Available columns: {list(df.columns)}")

df.head()

,OPERATION_ID,LOCATION,ROOM,BOOKING_DATE,CASE_STATUS,OPERATION_TYPE,EMERGENCY_PRIORITY,PLANNED_PATIENT_CALL_TIME,PLANNED_PATIENT_FETCH_TIME,PLANNED_RECEPTION_IN_TIME,...,ADMISSION_BED,AOH,BLOOD,IMPLANT,DIAGNOSIS,CANCER_INDICATOR,TRAUMA_INDICATOR,Delay_Reason,Remarks,PATIENT_CODE_OLD
0,588456,Main Building OT,MBOR11,10:27:00,Final,Elective,0,2019-04-11 09:50:00,2019-04-11 09:50:00,2019-04-11 09:50:00,...,N06A036,false,NIL,required microscope,right breast ca,false,false,Surgeon (e.g. Surgeon not available & etc.),0,i8fr0NKEp
1,590736,Main Building OT,MBOR05,14:50:00,Final,Elective,0,2019-04-11 10:40:00,2019-04-11 10:40:00,2019-04-11 10:40:00,...,NASWA11,false,NIL,0,early pregnancy failure,false,false,0,0,So30dGuVZ
2,591995,ICL,RoomC,18:30:00,Actualised,Elective,0,2019-04-11 10:55:00,2019-04-11 10:55:00,2019-04-11 10:55:00,...,N07B005,false,NIL,0,not recorded,false,false,0,0,0t2jmERej
3,590451,Main Building OT,MBOR04,11:49:00,Final,Elective,0,2019-04-11 10:50:00,2019-04-11 10:50:00,2019-04-11 10:50:00,...,N041004,false,NIL,need eustachian tube ballon,eustachian tube disorder,false,false,0,0,wDgw8UGdT
4,573666,Medical Center OT,MCOR03,09:00:00,Final,Elective,0,2019-04-11 10:30:00,2019-04-11 10:30:00,2019-04-11 10:30:00,...,N02A025,false,NIL,0,gall bladder stone,false,false,first case havent finished,first case havent finished,WYQEJDtnk


Normalize text in Delay_Reason (remove punctuation, standardise case, remove trialing spaces)

In [3784]:
_punct_tbl = str.maketrans("", "", string.punctuation)

def normalize_text(s: str) -> str:

    s = str(s).lower()
    s = s.translate(_punct_tbl)
    s = re.sub(r"\s+", " ", s).strip()

    s = re.sub(r"\bo\.t\b", "operating theater", s)
    s = re.sub(r"\bot\b", "operating theater", s)
    s = re.sub(r"\bo\.r\b", "operating room", s)
    s = re.sub(r"\banaesth\b", "anaesthesia", s)
    s = re.sub(r"\banesth\b", "anaesthesia", s)
    s = re.sub(r"\bpt\b", "patient", s)
    s = re.sub(r"\bprev\b", "previous", s)
    s = re.sub(r"\bdr\b", "doctor", s)
    s = re.sub(r"\bpre-med\b", "premedication", s)

    return s

# Apply normalization
df["_Delay_norm"] = df[DELAY_COL].astype(str).fillna("").map(normalize_text)
df[["_Delay_norm"]].head(10)

,_Delay_norm
0,surgeon eg surgeon not available etc
1,0
2,0
3,0
4,first case havent finished
5,surgeon eg surgeon not available etc
6,e case
7,0
8,0
9,0


In [3785]:
STOPWORDS = {
    "the","a","an","and","or","to","of","for","by","with","from",
    "is","are","was","were","be","been","being","due","because",
    "this","that","it","as","into","per","via", "eg", "etc"
}

# Initialize containers
words, bigrams, trigrams = [], [], []

# Tokenize each delay reason
for text in df["_Delay_norm"]:
    tokens = [t for t in text.split() if t and t not in STOPWORDS]
    if not tokens:
        continue

    words.extend(tokens)
    if len(tokens) >= 2:
        bigrams.extend([" ".join(tokens[i:i+2]) for i in range(len(tokens)-1)])
    if len(tokens) >= 3:
        trigrams.extend([" ".join(tokens[i:i+3]) for i in range(len(tokens)-2)])

word_counts    = Counter(words)
bigram_counts  = Counter(bigrams)
trigram_counts = Counter(trigrams)


In [3786]:
pd.DataFrame(word_counts.most_common(),   columns=["Word",   "Count"]).to_csv(OUT_WORDS,   index=False, encoding="utf-8-sig")
pd.DataFrame(bigram_counts.most_common(), columns=["Bigram", "Count"]).to_csv(OUT_BIGRAMS, index=False, encoding="utf-8-sig")
pd.DataFrame(trigram_counts.most_common(),columns=["Trigram","Count"]).to_csv(OUT_TRIGRAMS,index=False, encoding="utf-8-sig")

print(f"Saved:\n - {OUT_WORDS}\n - {OUT_BIGRAMS}\n - {OUT_TRIGRAMS}")

Saved:
 - oots-delay-wordcount-cleaned-1.csv
 - oots-delay-bigram-count-cleaned-1.csv
 - oots-delay-trigram-count-cleaned-1.csv


Print top, words, bigrams, trigrams (change file according to updates)

In [3787]:
top_n = 30

print("\nTop Words:")
display(pd.DataFrame(word_counts.most_common(top_n), columns=["Word", "Count"]))

print("\nTop Bigrams:")
display(pd.DataFrame(bigram_counts.most_common(top_n), columns=["Bigram", "Count"]))

print("\nTop Trigrams:")
display(pd.DataFrame(trigram_counts.most_common(top_n), columns=["Trigram", "Count"]))



Top Words:


,Word,Count
0,0,204532
1,case,15212
2,surgeon,11591
3,registration,9880
4,admin,9872
5,not,9013
6,am,7546
7,available,6612
8,list,6508
9,last,6345



Top Bigrams:


,Bigram,Count
0,admin registration,9870
1,not available,6565
2,elective list,6141
3,last scheduled,6122
4,case day,6122
5,surgeon not,5403
6,surgeon surgeon,5381
7,e case,4661
8,resequencing cases,4579
9,mcot end,4424



Top Trigrams:


,Trigram,Count
0,surgeon not available,5398
1,surgeon surgeon not,5380
2,end am elective,3726
3,am elective list,3726
4,mcot end am,3722
5,last scheduled am,3722
6,scheduled am case,3722
7,am case day,3722
8,elective list 830am1pm,3437
9,list 830am1pm last,3437


In [3788]:
import numpy as np

INPUT_FILE  = "oots-data-cleaning-1.xlsx"
OUTPUT_FILE = "oots-data-cleaning-3-flagged.xlsx"
OUTPUT_CSV  = "oots-data-cleaning-3-flagged.csv"

df = pd.read_excel(INPUT_FILE)

COL = "Delay_Reason"
s = df[COL].astype(str)

clean = (
    s.str.lower()
     .str.replace(r"[^\w\s]", "", regex=True)    
     .str.replace(r"\s+", " ", regex=True)       
     .str.strip()
)

raw = s.str.strip()
only_punct_or_numbers = raw.str.match(r'^(?=.*\S)(?!.*[A-Za-z]).*$', na=False)

df.loc[only_punct_or_numbers, COL] = "0"

not_late_phrases = [
    "no delay", "not delay", "not delayed", "not late",
    "na", "0", "null", "nan"
]

def phrase_to_token_pattern(p: str) -> str:
    p = p.strip().lower()
    esc = re.escape(p).replace(r"\ ", r"\s+")
    return rf"(?<!\w){esc}(?!\w)"

pattern = r"(?:{})".format("|".join(phrase_to_token_pattern(p) for p in not_late_phrases))
regex = re.compile(pattern, flags=re.IGNORECASE)

phrase_hit = clean.str.contains(regex, na=False)

df["Reason_Is_Late"] = np.where(only_punct_or_numbers | phrase_hit, 0, 1)

df[COL] = clean
df.loc[only_punct_or_numbers, COL] = "0"

# Save
df.to_excel(OUTPUT_FILE, index=False)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("Saved:")
print(f" - {OUTPUT_FILE}")
print(f" - {OUTPUT_CSV}")
print(df[[COL, "Reason_Is_Late"]].head(20))

Saved:
 - oots-data-cleaning-3-flagged.xlsx
 - oots-data-cleaning-3-flagged.csv
                                         Delay_Reason  Reason_Is_Late
0                surgeon eg surgeon not available etc               1
1                                                   0               0
2                                                   0               0
3                                                   0               0
4                          first case havent finished               1
5                surgeon eg surgeon not available etc               1
6                                              e case               1
7                                                   0               0
8                                                   0               0
9                                                   0               0
10                                                  0               0
11                                                  0               0
12        

## Encode Categorical Variables